In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests 
import time
import re

# pip install geopy
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from geopy_functions import *
from my_functions import *

In [135]:
# import the dataframes
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv', keep_default_na=False)  # albums with >= 20 votes, mostly from rock, worldwide
df_coordinates_final = pd.read_csv('Datasets/df_coordinates_final.csv')               # merge of df_masters_blended with their locations
df_coordinates_us = pd.read_csv('Datasets/df_coordinates_us.csv')
df_coordinates_uk = pd.read_csv('Datasets/df_coordinates_uk.csv')        
df_final_uk = pd.read_csv('Datasets/other_datasets/df_final_uk.csv')
df_final_us = pd.read_csv('Datasets/other_datasets/df_final_us.csv')
df_artist_countries = pd.read_csv('Datasets/other_datasets/df_artist_countries.csv')
df_artists_origins = pd.read_csv('Datasets/other_datasets/df_artists_origins.csv')
df_artists_origins_coordinates = pd.read_csv('Datasets/other_datasets/df_artists_origins_coordinates.csv')
df_ratings = pd.read_csv('Datasets/df_ratings.csv')

df_exploded = pd.read_csv('Datasets/df_exploded.csv')
df_final = pd.read_csv('Datasets/df_final.csv')
df_country = pd.read_csv('Datasets/other_datasets/df_country.csv')

# print information
print(f'{df_ratings_20.shape}: df_ratings_20')
print(f'{df_coordinates_final.shape}: df_coordinates_final')
print(f'{df_ratings.shape}: df_ratings')
print(f'{df_final.shape}: df_final')
print(f'{df_exploded.shape}: df_exploded')
print(f'{df_artist_countries.shape}: df_artist_countries')


(51222, 5): df_ratings_20
(5634, 7): df_coordinates_final
(11067, 9): df_ratings
(16781, 10): df_final
(16795, 10): df_exploded
(11065, 2): df_artist_countries


In [137]:
df_coordinates_uk

,artist,country,city,latitude,longitude,address,state
0,Korpse,United Kingdom,Aberdeen,57.142649,-2.097635,"Aberdeen, Guild Street, City Centre, Aberdeen ...",Alba / Scotland
1,Pallas,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
2,The Xcerts,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
3,Annie Lennox,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
4,Hellripper,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
...,...,...,...,...,...,...,...
1490,Asking Alexandria,United Kingdom,York,53.959055,-1.081536,"York, York and North Yorkshire, England, YO1 8...",England
1491,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
1492,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
1493,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire


In [136]:
df_coordinates_uk.sort_values('population', ascending=False)

KeyError: 'population'

In [25]:
df_exploded[df_exploded['rating']==3.57].query("year == 2004")

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style
6866,2004,Joan of Arc,"Joan of Arc, Dick Cheney, Mark Twain...",3.57,23,57.27,17,3.37,indie,Indie Rock
11586,2004,Seemless,Seemless,3.57,36,50.15,12,4.18,metal,Hard Rock
11587,2004,Seemless,Seemless,3.57,36,50.15,12,4.18,metal,Stoner Rock
11945,2004,Skywave,Synthstatic,3.57,23,47.52,14,3.39,indie,Shoegaze
14747,2004,The Showdown,A Chorus of Obliteration,3.57,98,42.48,10,4.25,metal,Heavy Metal
14748,2004,The Showdown,A Chorus of Obliteration,3.57,98,42.48,10,4.25,metal,Thrash


In [3]:
df_coordinates_final[df_coordinates_final['artist']=='Skywave']

,artist,country,city,latitude,longitude,address,state


In [4]:
df_artists_origins[df_artists_origins['artist']=='Skywave']

,artist,origin
5699,Skywave,"Fredericksburg, VA, United States"


# Locations Wikipedia scraping

## Import dataframes

In [177]:
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv', keep_default_na=False)

print(f"{df_ratings_20['artist'].nunique()} artists")
print(f"{df_ratings_20.shape[0]} albums")
print(f"Average of {round(df_ratings_20.shape[0] / df_ratings_20['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 20 votes")

14840 artists
51222 albums
Average of 3.45 albums per artist in the subset with the (mostly UK) albums with more than 20 votes


In [178]:
df_ratings_20.shape

(51222, 5)

## Testing

In [28]:
df.sample(5)

,artist,title,year,album_length,tracks,genres,styles,release_country,artist_profile
10580,The 1975,A Brief Inquiry into Online Relationships,2018,58.43,15,"['Rock', 'Pop']","['Indie Rock', 'Alternative Rock', 'Indie Pop']",UK & Europe,British indie rock band. \r\n\r\nPop-rock band...
9889,Le Butcherettes,A Raw Youth,2015,0.00,12,['Rock'],[],US,Formed by Teri Gender Bender and Auryn Jolene ...
6394,John Fogerty,Centerfield,1985,35.33,9,['Rock'],"['Pop Rock', 'Folk Rock', 'Country Rock']",US,"American musician, songwriter, and guitarist (..."
4498,L7,The Beauty Process: Triple Platinum,1997,41.57,12,['Rock'],"['Punk', 'Grunge']",US,American grunge punk/alternative rock band fro...
5742,The Fall,Are You Are Missing Winner,2001,47.68,10,['Rock'],"['Garage Rock', 'Punk', 'Rockabilly']",UK,"Post-punk band from Greater Manchester, UK. 19..."


In [183]:
artists = df_uk_masters['artist'].unique()
len(artists)

27045

In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [658]:
artist = "John Grant"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
name_changed_musician = name_changed + ('_(musician)')

url = f"https://en.wikipedia.org/wiki/{name_changed_musician}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the musician is dead
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Genres" in text:
                        location = text.split('Genres')[0]
                        print('Genres')
                    else:
                        if "Occupations" in text:
                            location = text.split('Occupations')[0]
                            print('Occupations')
                        else:
                            location = np.nan
        except:  
            location = np.nan
except:
    print('fuck')


Genres


In [659]:
location

'Buchanan, Michigan, U.S.'

In [652]:
name_changed_musician

'John_Grant_(musician)'

In [533]:
artist = "Stone Gossard"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')

url = f"https://en.wikipedia.org/wiki/{name_changed}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    location = table[0].text.split('Origin')[1].split('Genres')[0]

# save info in lists
    print('origin')

except:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the artist is dead
        print('dead')
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Occupations" in text:
                        location = text.split('Occupations')[0]
                    else:
                        if "Genres" in text:
                            location = text.split('Genres')[0]
                            print(repr(location))
                        else:
                            location = np.nan
        except:  
            location = np.nan


In [535]:
text

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metalOccupationsMusiciansongwriterInstrumentsGuitarvocalsYears active'

In [536]:
location

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metal'

## Changing dataframes

### **``df_artists_origins``**

In [93]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins.shape

(3251, 2)

In [94]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [95]:
df_artists_origins[df_artists_origins['origin']=='United Kingdom']

,artist,origin


In [86]:
df_new_artists = df_artists_origins[df_artists_origins['origin']=='United Kingdom']

In [89]:
df_new_artists

,artist,origin
1059,Son of Dork,"London, England"
1192,Mojave 3,"London, England"
1326,Crippled Black Phoenix,"Bristol, England"
1597,This Mortal Coil,"Wandsworth, England"
2191,Arcadia,"London, England"
2300,Jade Warrior,"London, England"
2348,The Waterboys,"London, England"
2469,Blackmore's Night,"Mount Sinai, NY, United States"
2556,Atomic Rooster,"London, England"
2609,The Nefilim,"Lambeth, England"


In [88]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Son of Dork", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Mojave 3", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Crippled Black Phoenix", "Bristol, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="This Mortal Coil", "Wandsworth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Arcadia", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jade Warrior", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Waterboys", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Blackmore's Night", "Mount Sinai, NY, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Atomic Rooster", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Nefilim", "Lambeth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Spiders", "Sheffield, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brontide", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gilgamesh", "Hampstead, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Young Legionnaire", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Deviants", "Ladbroke Grove, London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Head of David", "Dudley, West Midlands, United Kingdom", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Quintessence", "London, England", df_new_artists["origin"])

In [53]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[8016]['artist_profile']
splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Formed in 1969, they played a blend of jazz, progressive rock, Indian Music, and new age rock.   Members included:
Sambhu Babaji : Bass  Dave Codling : Guitar  Shiva Shankar Jones : Keyboards, Vocals  Jake Milton : Drums  Alan Mostert
: Guitar  Raja Ram : Flute, Piano, Vocals


In [52]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Quintessence".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
8016,1969,Quintessence,In Blissful Company,3.88,16,0.0,8,['Psychedelic Rock'],UK,"Formed in 1969, they played a blend of jazz, p..."


In [90]:
df_artists_origins = df_artists_origins[df_artists_origins['origin']!='United Kingdom']
df_artists_origins.shape

(3234, 2)

In [91]:
df_artists_origins.to_csv('Datasets/df_artists_origins.csv', index=False)

In [92]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3251, 2)


In [56]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/9 - London, Greater London, England, United Kingdom
2/9 - Bristol, City of Bristol, West of England, England, United Kingdom
3/9 - Wandsworth, London Borough of Wandsworth, London, Greater London, England, SW18 1UJ, United Kingdom
4/9 - Mount Sinai, Miller Place, Town of Brookhaven, Suffolk County, New York, 11766, United States
5/9 - Lambeth, London Borough of Lambeth, London, Greater London, England, SE1 7JW, United Kingdom
6/9 - Sheffield, South Yorkshire, England, United Kingdom
7/9 - Hampstead, Greater London, England, NW3 1QG, United Kingdom
8/9 - Ladbroke Grove, Westway, Lancaster West Estate, North Kensington, Royal Borough of Kensington and Chelsea, London, Greater London, England, W10 5YG, United Kingdom
9/9 - Dudley, West Midlands, England, United Kingdom


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Bristol,"Bristol, England","Bristol, England",51.453802,-2.597298,"Bristol, City of Bristol, West of England, Eng..."
1,United Kingdom,Dudley,"Dudley, West Midlands, United Kingdom","Dudley, West Midlands, United Kingdom",52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
2,United Kingdom,Hampstead,"Hampstead, England","Hampstead, England",51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3,United Kingdom,Ladbroke Grove,"Ladbroke Grove, London, England","Ladbroke Grove, London, England",51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
4,United Kingdom,Lambeth,"Lambeth, England","Lambeth, England",51.495211,-0.116335,"Lambeth, London Borough of Lambeth, London, Gr..."
5,United Kingdom,London,"London, England","London, England",51.507446,-0.127765,"London, Greater London, England, United Kingdom"
6,United Kingdom,Sheffield,"Sheffield, England","Sheffield, England",53.380663,-1.470228,"Sheffield, South Yorkshire, England, United Ki..."
7,United Kingdom,Wandsworth,"Wandsworth, England","Wandsworth, England",51.457027,-0.193261,"Wandsworth, London Borough of Wandsworth, Lond..."
8,United States,Mount Sinai,"Mount Sinai, NY, United States","Mount Sinai, NY, United States",40.941066,-73.019455,"Mount Sinai, Miller Place, Town of Brookhaven,..."


### **``df_coordinates``**

In [57]:
df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scraped.shape

(1527, 7)

In [58]:
df_coordinates_scraped.head()

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Adelaide,"Adelaide, South Australia, Australia","Adelaide, South Australia, Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
1,Australia,Adelaide,"Adelaide, South Australia","Adelaide, South Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
2,Australia,Blue Mountains,"Blue Mountains, NSW, Australia","Blue Mountains, NSW, Australia",-33.609741,150.405224,"Blue Mountains, New South Wales, Australia"
3,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
4,Australia,Byron Bay,"Byron Bay, New South Wales, Australia","Byron Bay, New South Wales, Australia",-28.648333,153.617778,"Byron Bay, Byron Shire Council, New South Wale..."


In [59]:
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address
702,United Kingdom,United Kingdom,United Kingdom,United Kingdom,54.702354,-3.276575,United Kingdom


In [60]:
df_coordinates_scraped.drop(702, axis=0, inplace=True)
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address


In [62]:
df_coordinates_scraped.to_csv('Datasets/df_coordinates.csv', index=False)

In [63]:
df_coordinates.shape

(9, 7)

In [64]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1526, 7)

Found 4 duplicates:
               city         country
324         Bristol  United Kingdom
515  Ladbroke Grove  United Kingdom
542          London  United Kingdom
650       Sheffield  United Kingdom

Resulting dataset: (1531, 7)
Merged artists with coordinates! Found 5 new locations
df_coordinates_concat exported to .csv


### **``df_artists_origins_coordinates_concat``**

In [74]:
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3246,Gilgamesh,United Kingdom,Hampstead,51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3247,Young Legionnaire,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
3248,The Deviants,United Kingdom,Ladbroke Grove,51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
3249,Head of David,United Kingdom,Dudley,52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
3250,Quintessence,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


## Changing values

In [72]:
df_masters_blended[df_masters_blended['artist'].str.contains('Nucleus')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1004,3821235,1537522,13529787,US,Nucleus (US),Entity,2019,38.42,8,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
1592,3821235,1094310,8362817,US,Nucleus (US),Sentient,2016,37.92,9,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
5257,184256,175620,279855,UK,Nucleus (UK),We'll Talk About It Later,1971,0.00,7,"['LP', 'Album']","['Jazz', 'Rock']","['Fusion', 'Jazz-Funk', 'Jazz-Rock', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."
10195,184256,23574,465143,UK,Nucleus (UK),Elastic Rock,1970,0.00,13,"['LP', 'Album']","['Jazz', 'Rock']","['Jazz-Rock', 'Fusion', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."


In [75]:
df_masters_blended[df_masters_blended['title'].str.contains('Alleycat')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [77]:
df_ratings_20[df_ratings_20['artist'].str.contains('Nucleus')]

,album_id,artist,title,rating,votes
29004,86398,Nucleus (UK),Elastic Rock,3.55,20
29120,87140,Nucleus (UK),We'll Talk About It Later,3.79,21
39636,216509,Nucleus (US),Sentient,3.29,29
45524,333839,Nucleus (US),Entity,3.68,40
46380,352917,Nucleus (UK),Alleycat,3.52,20


In [76]:
df_ratings_20.loc[46380, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29004, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29120, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[39636, 'artist'] = 'Nucleus (US)'
df_ratings_20.loc[45524, 'artist'] = 'Nucleus (US)'

In [869]:
df.drop([7660, 8037], axis=0, inplace=True)
df.shape

(12059, 10)

In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [78]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [581]:
df.shape

(12059, 10)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## Defining the functions

In [15]:
def get_origins_wikipedia(df, start_index, final_index):
    df = pd.read_csv('Datasets/df_rock_ratings_20.csv')
    artists = df['artist'].unique()

    try:
    # import the DataFrame with the locations whose coordinates I already have
        df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
        print('Bingo! df_coordinates.csv found \n')
    except: 
        print('df_coordinates.csv not found \n')

    artists_list = []
    origin_list = []
    count=0
    scraped=0

    for index in artists_us_to_do[start_index:final_index]:

        name_changed = index.replace(' ', '_')
        name_changed_band = name_changed + ('_(band)')

        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            location = table[0].text.split('Origin')[1].split('Genres')[0]
            city = location.split(', ')[0]
            count+=1
            
        # save info in lists
            artists_list.append(index)  
            origin_list.append(location)
            scraped+=1
            print(f'{scraped}/{count} - {name_changed_band}: {location}')

        except:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")
                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                try:
                    location = table[0].text.split('Origin')[1].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1 
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed}: {location}')

                except:
                    location = table[0].text.split(')')[2].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1

                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (individual): {location}')

            except:
                try:
                    url = f"https://es.wikipedia.org/wiki/{name_changed}"
                    response = requests.get(url).content
                    soup = BeautifulSoup(response, "html.parser")

                    table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')
                    location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                    city = location.split(', ')[0]
                    count+=1    
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (español): {location}')

                except:
                    count+=1
                    print(f'{scraped}/{count} - {index}: error')
                    artists_list.append(index) 
                    origin_list.append(np.nan)

        if len(artists_list) != len(origin_list):
            print('different lengths')
            break

    df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
    
    return df_artists_origins

In [102]:
def get_new_artists(df_artists_origins):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

    if df_artists_origins['origin'].isna().sum() == 0:        
        print("No null values, but let's take a look just in case there are weird locations")

    else: 
    # take a look at the df with the new artists and make sure there are non null values in origin (when it couldn't find it in Wikipedia)
        print(f'{round(df_artists_origins['origin'].isna().sum() / df_artists_origins.shape[0]*100, 2)} % of nulls')
    
# subset of the new artists I just got, wether there are null values or not
    df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]

    print("Here is the dataframe with the new artists, without duplicates")
    return df_new_artists   # so I can take a look at it and then continue

In [ ]:
def export_artists_origins_concat(df_new_artists):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

# concat with the df I just got
    df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
    df_artists_origins_concat.drop_duplicates(inplace=True)     # just in case
    df_artists_origins_concat.reset_index(drop=True, inplace=True)

# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
    df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)
    print('df_artists_origins_concat exported to .csv')
    print(df_artists_origins_concat.shape)

In [175]:
def get_coordinates_geopy(df_new_artists):
    
# replace special characters for spaces
    df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
    df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# run the function that gets the coordinates from the origins from Geopy
    geolocator = Nominatim(user_agent="music_analysis", timeout=10)

# if they are 'dirty' origins that after the cleaning, they result in the same 'origin_clean'
    df_unique = df_new_artists[['origin', 'origin_clean']].drop_duplicates() 
    unique_origins = df_unique['origin'].values
    unique_origins_clean = df_unique['origin_clean'].values

    country_list = []
    city_list = []
    latitude_list = []
    longitude_list = []
    address_list = []
    lists = [country_list, city_list, latitude_list, longitude_list, address_list]
    count = 0

    for origin in unique_origins_clean:
        count+=1
        time.sleep(1)
        location = geolocator.geocode(origin)

        print(f'{count}/{len(unique_origins_clean)} - {location.address}')  

    # save the info in lists
        country_list.append(location.address.split(', ')[-1])
        city_list.append(origin.split(', ')[0])
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
        address_list.append(location.address)

        # # Check lengths
        # print(f"{count}/{len(unique_origins_clean)} - {origin}")
        # print(f"Current list lengths -> country: {len(country_list)}, city: {len(city_list)}, "
        #     f"lat: {len(latitude_list)}, lon: {len(longitude_list)}, address: {len(address_list)}")

    df_coordinates = pd.DataFrame({'country': country_list
                                , 'city': city_list
                                , 'origin': unique_origins
                                , 'origin_clean': unique_origins_clean
                                , 'latitude': latitude_list
                                , 'longitude': longitude_list
                                , 'address': address_list})
    df_coordinates.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates.reset_index(drop=True, inplace=True)

    return df_coordinates

In [ ]:
def export_coordinates_concat(df_coordinates):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
    print(f"df_coordinates_scraped: {df_coordinates_scraped.shape}\n")

# concat with the df of the coordinates I just got
    df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
    df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates_concat.reset_index(drop=True, inplace=True)

# look for duplicates in the origin, between the locations I had already gotten and the new ones
    check_duplicates_origins(df_coordinates_concat)
    new_origins = df_coordinates_concat.shape[0] - df_coordinates_scraped.shape[0]
    print(f"Merged artists with coordinates! Found {new_origins} new locations")

# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
    df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)
    print('df_coordinates_concat exported to .csv')

In [140]:
def merge_origins_coordinates(df_new_artists):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
    df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
    df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

# import the df that contains info of the artists and the coordinates of their origins
    df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# concat to get the df with all the artists, origins and their coordinates
    df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
    df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)

# save it in a csv file
    df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)
    print("Exported to a .csv file")

    return df_artists_origins_coordinates_concat

## Merge dataframes and look for the ``new_artists``

In [11]:
artists_blend = df_masters_blended['artist'].unique()
len(artists_blend)

4527

In [12]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_usa = []

for artist in artists_blend:
    if artist not in df_artists_origins['artist'].values:
        artists_usa.append(artist)

len(artists_usa)

1555

In [13]:
artists_usa

['As Living Arrows',
 'Hidden Mothers',
 'Tiny Moving Parts',
 'Poppy',
 'State Champs',
 'Oso Oso',
 'Better Lovers',
 'Lowen',
 'Halsey',
 'Amyl and the Sniffers',
 'Delta Sleep',
 'High Vis',
 'Cemetery Skyline',
 'Goat',
 'Chat Pile',
 'Drug Church',
 'Origami Angel',
 'Heriot',
 'Nightwish',
 'Foxing',
 'Alora Crucible',
 'Wage War',
 'TURQUOISEDEATH',
 'Dawn Treader',
 'Boston Manor',
 'MJ Lenderman',
 'Fat Dog',
 'Kingcrow',
 'Leprous',
 'Wunderhorse',
 'thrown',
 'Horse Jumper of Love',
 'Within the Ruins',
 'Magdalena Bay',
 'Fontaines D.C.',
 'beabadoobee',
 'State Faults',
 'Graphic Nature',
 'The Home Team',
 'Speed',
 'Remi Wolf',
 'Clairo',
 'Crippling Alcoholism',
 'Cigarettes After Sex',
 'Abriction',
 'Pijn',
 'Outlander',
 'Imagine Dragons',
 'The Dangerous Summer',
 'Pond',
 'Hyperdontia',
 'Vredehammer',
 'Pedro the Lion',
 'Weston Super Maim',
 'Stand Still',
 'Mortal Wound',
 'Eye of Solitude',
 'Beth Gibbons',
 'Contention',
 'Knocked Loose',
 'Mdou Moctar',
 'Th

In [110]:
# I have to import this df for the function to properly work
df = pd.read_csv('Datasets/df_blend_ratings.csv')
artists = df['artist'].unique()
len(artists)

4249

In [11]:
artists[-1]

'As Living Arrows'

In [12]:
df_artists_origins.index[-1]

3423

In [13]:
df_artists_origins.tail()

,artist,origin
3419,Cinderella,"Philadelphia, Pennsylvania, U.S."
3420,Death From Above 1979,"Toronto, Ontario, Canada"
3421,Chicago,"Chicago, Illinois, United States"
3422,The Nation of Ulysses,"Washington, D.C."
3423,Strung Out,"Simi Valley, California, U.S."


In [164]:
np.where(artists=='July Talk')

(array([2772], dtype=int64),)

In [163]:
artists[2772]

'July Talk'

# **Code to execute the functions from ``geopy_functions.py``**

In [5]:
artists_to_remove = ['Cemetery Skyline', 'Goat', 'Kingcrow', 'Speed', 'Hyperdontia', 'Vredehammer', 'Weston Super Maim',
                'Mdou Moctar', 'AVRALIZE', 'Engulfed', 'Coffin Storm', 'samlrc', 'Little Kid', 'Termina', 'Rorcal',
                'Reverence To Paroxysm', 'Voyager', 'Blindfolded and Led to the Woods', 'Mork', 'Yeule', 'Pond',
                'Empire State Bastard', 'Blood Command', 'Avatar', 'Brutus', 'Faceless Burial', 'Within Destruction',
                'Sedimentum', 'Ataraxy', 'Corpsessed', 'Epitaphe', 'Aldous Harding', 'Messa', 'Ghost', 'Ecchymosis',
                'Wormrot', 'Vertebra Atlantis', 'Papangu', 'Sermon of Flames', 'Springtime', 'Monolord', 'Sulphurous',
                'LLNN', 'Trna', 'Slaughter To Prevail', 'sonhos tomam conta', 'Galvanizer', 'Ophidian I', 'Diabolizer',
                'Morbific', 'Defacement', 'Divide And Dissolve', 'Soen', 'Scorpions', 'Accept', 'Respire', 'Undergang',
                'Bearings', 'Scalp', 'Miasmatic Necrosis', 'Teenage Mutant Ninja Turtles', 'Gorephilia', 'Vous Autres',
                'Carnation', 'Pharmacist', 'Paara', "Justice For The Damned", 'VVilderness', 'Molested Divinity', 'Ellis',
                'Kontinuum', 'Monsters', 'Giver', 'Lowrider', 'Vengeful Spectre', 'Vomit the Soul', 'Sadisme', 'Alarmist',
                'Klone', 'Nocturnal Departure', 'King Gizzard and The Lizard Wizard', 'Make Them Suffer', 'The Chats',
                'Patrick Watson', 'Shirokuma', 'Forests', 'Town Portal', 'Ceremony Of Silence', 'CHAI', 'Baalsebub',
                'Minors', 'Mono', 'Tallies', 'Normandie', 'Mouse On The Keys', 'Burial Invocation', 'Orville Peck',
                'Lunatic Soul', 'Alex Lahey', 'Hozier', 'Mystifier', 'Hands Like Houses', 'Ruins', 'Autokrator',
                'Legend of the Seagullmen', 'Death Toll 80k', 'IDYLLS', 'Spaceslug', 'i hate sex', 'Band-Maid',
                'With the Dead', 'Hungry Ghosts', 'Middle Kids', 'Gleb Kolyadin', "Leaves' Eyes", "Phrenelith",
                "David Brent", "Art Of Dying", "Minor Victories", "Pantha Du Prince", "Schammasch", 'LUH',
                'Violet Cold', 'EL VY', 'Shining', 'Hills', "Mutiny On The Bounty", 'Lucifer', 'FFS', 'Ranger',
                'Trepalium', 'A Textbook Tragedy', 'This Is A Standoff', 'FACT', 'Swallowed', 'Disembarked',
                'Wildbirds and Peacedrums', 'Archivist', 'Timber Timbre', 'Newsboys', 'Dope Lemon', 'Vagabon',
                'RY X', 'Moonlit Sailor', 'The Drones', 'Sarah Neufeld', 'Say Lou Lou', 'Cruciamentum', 'Lay Down Rotten',
                'Dreamshade', 'Sickening Gore', 'Circles', "Avey Tare's Slasher Flicks", 'Forest Silence',
                "One Eyed God Prophecy", 'Coffins', 'Osamu Kitajima', 'Living With Lions', 'Ansur', 'Parades',
                "Intestine Baalism", 'Comity', 'No Omega', 'Wolverine', 'Disavowed', 'Angel Dust', "!T.O.O.H.!",
                'Hypnosia', 'Hexenhaus', 'Paradox', 'Deathrow', 'Excruciate', 'FareWell Poetry', 'Sights and Sounds',
                'Supersister', "Birds Of Tokyo", 'Ark', "The Flower Kings", 'Beardfish', 'Graveworm', 'Acid',
                'Ladyhawke', 'Geddy Lee', 'Yngwie Malmsteen', "World's End Girlfriend", 'Totem Skin', 'Lewis',
                'I Hate Sally', "The Band", 'Lisa Hannigan', 'Lethal', 'Bubu', 'Van She', 'Mooncake', 'The Haunted',
                "Orphaned Land", 'Madder Mortem', 'Kataxu', 'Gilberto Gil', 'Vendetta', 'Kvist', 'Acrostichon', 'Pain',
                'Obliteration', 'Flames of Hell', 'Wombbath', 'Stone', 'Disgrace', 'Fionn Regan', 'Disastrous Murmur',
                'Urfaust', 'Sleepingdog', 'Island', 'Bethlehem', 'Subterranean Masquerade', 'After Dinner', 
                'Black Boned Angel', 'FM', 'Embrace', 'Solefald', 'Maneige', 'Amberian Dawn', 'OOIOO', 'Anekdoten',
                "Aphrodite's Child", 'Hollenthon', 'Lykke Li', 'Lenka', 'Sarah McLachlan', 'Owen Pallett',
                'Devin Townsend Project', 'Missy Higgins', 'The Devin Townsend Band', 'Selda', 'Massacra', "Rory Gallagher",
                'Taste', 'Celestial Season', 'Ida Maria', 'Dark Tranquillity', 'Cadaver', 'Pele', 'Exuma',
                'Great Lake Swimmers', 'Dawn', 'The Bats', 'Yoko Ono', 'Illogicist', 'The Saints', 'Final Fantasy',
                'Pendulum', 'Lunar Aurora', 'Bee Gees', 'Stars', "David Sylvian and Robert Fripp", 'Afflicted', 'Lengsel',
                'Extol', 'MDFMK', 'Univers Zero', 'Mortuary Drape', 'Zyklon', 'Winds', 'Zyklon-B', 'The Sins of Thy Beloved',
                'Lords of Acid', 'Devin Townsend', 'Diablo Swing Orchestra', 'Arcturus', 'Cornelius', 'Manu Chao',
                'Bryan Adams', 'Peaches', 'Doro', 'Kingdom Come', 'Pekka Pohjola', 'Shakira', 'Massacre', 'Subhumans',
                'Set Fire to Flames', 'Gorgoroth', 'Gandalf', 'Klaus Schulze', 'The Ecstasy of Saint Theresa',
                "Lou Reed and John Cale", 'Brian Eno and David Byrne', 'Bob Dylan and The Band', 'Era', 'Devil Doll',
                'Cauterize', 'Roadrunner United', 'Circus Maximus', 'Crowpath', 'Raunchy', 'Tad Morose', 'Kenna',
                'Head Control System', 'Torchbearer', 'Rosesdead', 'Angtoria', 'Nightrage', 'Necros Christos', 'Hypnos 69',
                'Wold', 'Andromeda', 'Chad VanGaalen', 'Melechesh', 'Spheric Universe Experience', 'Anubis Gate',
                'The Project Hate MCMXCIX', 'Myrath', 'Savage Circus', 'Hartfield', 'Evereve', 'Daturah', 'Ad Infinitum',
                'Tash Sultana', 'Tarja Turunen', 'Ram-Zet', 'Sweven', 'Arcane', 'Sons of Apollo', 'Celesty', 'Brainstorm',
                'Unleash The Archers', 'Prostitute Disfigurement', 'The Psyke Project', 'Dornenreich', 'Watain', 'Funeral',
                'Cultes Des Ghoules', 'Revolting', 'Igorrr', 'Symfonia', 'Darkestrah', 'Sarah Blasko', 'Fractal Universe',
                'The End', 'Apotheosis', 'Drautran', 'Monolithe', 'CrazyEightyEight', 'Black Hole', 'Polaris', 'Nug',
                'Klabautamann', 'James LaBrie', 'Vance Joy', 'Helena Deland', 'Authorize', 'Blazon Stone', 'Rapture',
                'Worship', 'Conqueror', 'Swan Lake', 'Yyrkoon', 'Inquisition', 'Kerli', 'Keldian', 'Wake', 'Megiddo',
                'Juanes', 'William Hung', 'Votum', 'Atramentus', 'Abysmal Torment', 'Paul Dempsey', 'Cytotoxin',
                'Vulture Industries', 'Entrails', 'Jorn', 'Conception', 'Centaurus-A', 'Gezan', 'Blood Tsunami',
                'Machinemade God', 'Proscription', 'Ragnarok', 'Zeromancer', 'Satariel', 'Circle of Ouroborus', 'Emeth',
                'Before The Dawn', 'Near Death Condition', 'Ignivomous', 'MEANS', 'Svart Crown', 'Alan Sorrenti', 'Chthonic',
                'Blues Pills', 'Majestica', 'Bedsore', 'Ravencult', 'William Shatner', 'Cheval De Frise', 'Emptiness',
                'WHOURKR', 'Jet Black Stare', 'Despondency', 'October Falls', 'Istapp', 'Slumber', 'Night in Gales',
                'Ov HELL', 'Aficionado', 'Old Silver Key', 'Junior Battles', 'Adversarial', 'Shugo Tokumaru', 'North',
                'Fuck on the Beach', 'Anoice', 'Viscera', 'Two Tongues', 'The Last Felony', 'Revenge', 'The Secret',
                'Cosmic Putrefaction', 'Miseration', 'Azusa', 'Stalaggh', 'Cerebral Effusion', 'Nekromantheon', 'Blasphemer',
                'Pyaemia', 'Inveracity', 'Orchidectomy', 'Kraanium', 'Human Mincer', 'Deformity', 'Kaospilot',
                'Abominable Putridity', 'Iskra', 'Das Oath', 'Torsofuck', 'Internal Suffering', 'Nerlich', 'Blasphemophagher',
                'Head Wound City', 'Inanna', 'Disperse', 'Dishammer', 'Damaar', 'Extortion', 'Starring Janet Leigh',
                'The Arcane Order', "D'espairsRay", 'Demians', "Dave Matthews", 'Monsters of Folk', 'Shrinebuilder',
                'The Joe Perry Project', 'Boygenius', 'Scar the Martyr', 'Gordian Knot', 'Rock Star Supernova']

In [7]:
df_final = pd.read_csv('Datasets/df_final.csv')
unique_artists = df_final[df_final['year']<2011]['artist'].unique()

df_artists_origins = pd.read_csv('Datasets/other_datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_to_do = []

for artist in unique_artists:
    if artist not in artists and artist not in artists_to_remove:
        artists_to_do.append(artist)

len(artists_to_do)

0

In [4]:
# show the first next 5 artists I'm going to scrape
artists_to_do[:5]

['Grateful Dead']

In [5]:
# create the df with the origins scraped from Wikipedia

df = pd.read_csv('Datasets/df_final.csv')
start_index = 0
final_index = start_index+1

df_artists_origins = get_origins_wikipedia(df, start_index, final_index)

Bingo! df_coordinates.csv found 

1/1 - Grateful_Dead: Palo Alto, California, U.S.


In [6]:
df_artists_origins[df_artists_origins['artist']=='Grateful Dead']

,artist,origin
0,Grateful Dead,"Palo Alto, California, U.S."


In [7]:
# create a subset of the new artists I just got, and tell me if there are nulls
df_new_artists = get_new_artists(df_artists_origins)

# show the first new artists, if they were duplicates they have been dropped
df_new_artists.head()

No null values, but let's take a look just in case there are weird locations


,artist,origin
0,Grateful Dead,"Palo Alto, California, U.S."


### If there are null or weird values

In [14]:
# create a df with where origin is a null value
nulls = df_new_artists[df_new_artists['origin'].isna()]
nulls.shape

(0, 2)

**``np.where`` to replace the values for the real origins**

In [159]:
# so I can print the np.where and I save time
for artist in nulls['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "England", df_new_artists["origin"])')

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Math Horseman", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Broken Water", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Cire", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Arcane Order", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Reptilian", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Riot Before", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brother Von Doom", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Orchid", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Addaura", "E

**Looking in the internet for the real origins of these artists**

In [160]:
nulls.head(10) # so it's faster to copy the names of the artists to look for their origins on the internet

,artist,origin
192,Black Math Horseman,NaN
194,Broken Water,NaN
197,Cire,NaN
200,The Arcane Order,NaN
201,The Reptilian,NaN
206,The Riot Before,NaN
208,Brother Von Doom,NaN
209,Orchid,NaN
210,Addaura,NaN
215,Until Your Heart Stops,NaN


In [184]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Math Horseman", "Los Angeles", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Broken Water", "Olympia, Washington", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Cire", "New Orleans, LA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Reptilian", "Kalamazoo, Michigan", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Riot Before", "Richmond, VA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brother Von Doom", "Dayton, OH, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Orchid", "Amherst, Massachusetts", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Addaura", "Seattle, Washington", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Until Your Heart Stops", "Oakland, CA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Only Crime", "Fort Collins, CO, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="While Heaven Wept", "Dale City, VA, United States", df_new_artists["origin"])

In [178]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Only Crime	".strip()].sort_values('year').head()

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
5619,305973,386259,3227765,US,Only Crime,Virulence,2007,0.0,12,['Album'],['Rock'],['Punk'],Melodic hardcore band formed by [a=Good Riddan...


In [179]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[5619]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Melodic hardcore band formed by [a=Good Riddance] singer [a=Russ Rankin] and [a=Bane (2)] guitarist [a=Aaron Dalbec] in
2003.


#### Changing the original dataframes in case needed (ex: two bands with the same name)

In [73]:
df.loc[11090, 'artist'] = 'Embrace (US)'
df.loc[6140, 'artist'] = 'Embrace (UK)'
df.loc[6141, 'artist'] = 'Embrace (UK)'

In [74]:
df.to_csv('Datasets/df_masters_blended.csv', index=False)

In [75]:
# look for the albums of the artist in the original df_ratings_20 to check it's the correct artist
df_ratings_20[df_ratings_20['artist']=="Embrace".strip()]

,album_id,artist,title,rating,votes
4485,7197,Embrace,The Good Will Out,3.56,24
4486,7198,Embrace,Out Of Nothing,3.39,27
8649,14575,Embrace,Embrace,3.99,309


In [76]:
df_ratings_20.loc[8649, 'artist'] = 'Embrace (US)'
df_ratings_20.loc[4486, 'artist'] = 'Embrace (UK)'
df_ratings_20.loc[4485, 'artist'] = 'Embrace (UK)'

In [77]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [214]:
df_new_artists['origin'].isna().sum()

0

#### **Drop artists that are not from the UK or the US** (or supergroups that don't have a clear origin)

In [ ]:
artists_to_remove = ['Monsters of Folk', 'Shrinebuilder', 'The Joe Perry Project', 'Boygenius',
                     'Scar the Martyr', 'Gordian Knot', 'Rock Star Supernova']

for artist in artists_to_remove:
    try:
        artists_usa.remove(artist)
        print(f'{artist} removed')
    except:
        pass

In [193]:
# I can drop single rows or I can just create a subset with the artists that I want to keep, the ones that are not in the list of artists to remove
print(f"Rows before dropping: {df_new_artists.shape[0]}")
df_new_artists = df_new_artists[~df_new_artists['artist'].isin(artists_to_remove)]
print(f"Rows after dropping: {df_new_artists.shape[0]}")

Rows before dropping: 207
Rows after dropping: 206


#### **Check short and long origins, probably wrong**

In [187]:
# print abnormally short origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) < 10:
        print(index, row['origin'])

56 Fargo, ND
178 Boston


In [181]:
start_index = 207

end_index = start_index+5
df_new_artists.loc[start_index:end_index]

,artist,origin
207,While Heaven Wept,Virginia
208,Brother Von Doom,"Dayton, OH, United States"
209,Orchid,"Amherst, Massachusetts"
210,Addaura,"Seattle, Washington"
211,See You Next Tuesday,"Bay City, Michigan, United States"
212,Saints Never Surrender,"Fort Wayne, Indiana"


In [204]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Big Kids	".strip()].sort_values('year').head()

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
6396,152830,451305,2572483,US,Big Kids,Hoop Dreams,2010,0.0,8,['LP'],['Rock'],"['Rock & Roll', 'Punk']","Punk band from Oakland, CA, USA."


In [201]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[5587]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

American melodic hardcore punk band from Oakland, CA.


**Genres**

In [188]:
genres = df_new_artists[df_new_artists['origin'].str.contains('Genres')]
genres.shape

(7, 2)

**``np.where`` to replace the values for the real origins**

In [ ]:
for artist in genres['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "UK", df_new_artists["origin"])')

In [194]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Derek Sherinian", "Laguna Beach, California, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jeff Loomis", "Appleton, Wisconsin, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Johnny Hobo and the Freight Trains", "Brattleboro, Vermont, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bumblefoot", "Brooklyn, New York, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Ill Bill", "New York City, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Karl Sanders", "San Francisco, California, U.S.", df_new_artists["origin"])

In [195]:
# print the df so I can copy the origin from the column (before 'Genres')
df_new_artists[df_new_artists['origin'].str.contains('Genres')]

,artist,origin


In [206]:
# I create a new column to calculate the lenght of the origin, if it's long it probably didn't scrap correctly Wikipedia
df_new_artists["origin_length"] = df_new_artists["origin"].str.len()
long_strings = df_new_artists[df_new_artists["origin_length"]>40] # create a df based on these long origins
long_strings.shape # to see how many artists I have to take care of

(3, 3)

In [207]:
long_strings # display the df so I can copy the parts I am interested in

,artist,origin,origin_length
15,Latterman,"Huntington Station, New York, United States",43
147,Make Do and Mend,"West Hartford, Connecticut, United States",41
186,Baths,"Culver City, Los Angeles, California, United S...",51


In [ ]:
for artist in long_strings['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "UK", df_new_artists["origin"])')

In [ ]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Heartsounds", "Oakland, CA", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Burn Halo", "Costa Mesa", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gayngs", "Minneapolis, MN, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Sound Tribe Sector 9", "Snellville, GA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Big Kids", "Oakland, CA", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="SQRM", "Springfield, Massachusetts", df_new_artists["origin"])

In [209]:
# I drop the column I just created of 'origin_length'
df_new_artists = df_new_artists[['artist', 'origin']]

In [210]:
# bands from United Kingdom, wrong origin, poor level of detail
df_new_artists[df_new_artists['origin']==('United Kingdom')]

,artist,origin


### **Changing easy values: individuals that didn't get the right origin in Wikipedia**

In [119]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains(' and ')]

,artist,origin


In [61]:
# bands from Cumbria, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Cumbria')]

,artist,origin


In [62]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace('Cumbria', 'Cumberland'))

In [63]:
# bands from Middlesex, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Middlesex')]

,artist,origin


In [64]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Middlesex', ''))

In [65]:
# bands from Yorkshire, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('West Riding of Yorkshire')]

,artist,origin


In [66]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', West Riding of Yorkshire', ''))

In [67]:
# bands from Merseyside, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Merseyside')]

,artist,origin


In [68]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Merseyside', ''))

### **``np.where`` to replace the values for the real origins**

**Try a single origin in Geopy**

In [ ]:
# try to get the right origin of an origin that crashed
geolocator = Nominatim(user_agent="music_analysis")

origin = "Kirkland, Washington"

origin_clean = re.sub(r'\[\d+\]', '', origin).replace('.', '')
location = geolocator.geocode(origin_clean)
print(f"{location.address}")
print(location.latitude, location.longitude)

Kirkland, King County, Washington, United States
47.6765382 -122.2070775


In [212]:
start_index = 70

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
76,Artifex Pereo,"Louisville, Kentucky, U.S.[1][2]"
77,Midsummer,"Los Angeles, CA, United States"
78,Cold World,"Wilkes-Barre, PA"
79,SQRM,"Springfield, Massachussets"
80,Infest,"Valencia, California, U.S."


**Try all origins in Geopy**

In [8]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 0
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

0 - Palo Alto, Santa Clara County, California, United States


# **Export to .csv**

### GeoPy wrong locations

**In case GeoPy fails due to a wrong location, I have to delete the new locations, export again, change the location and run GeoPy again**

In [ ]:
#     # import the df with the artists' origins already scraped
# df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
# df_artists_origins_scraped = df_artists_origins_scraped[0:-20]
# df_artists_origins_scraped.to_csv('Datasets/df_artists_origins.csv', index=False)

In [9]:
df_new_artists.shape

(1, 2)

In [216]:
# in case I created by mistake already 'origin_clean' and I want to drop it
# df_new_artists = df_new_artists[['artist', 'origin']]
# df_new_artists

### **Export to .csv**

In [10]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(6738, 2)


## **GeoPy**

In [11]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/1 - Palo Alto, Santa Clara County, California, United States


,country,city,origin,origin_clean,latitude,longitude,address
0,United States,Palo Alto,"Palo Alto, California, U.S.","Palo Alto, California, US",37.444329,-122.159847,"Palo Alto, Santa Clara County, California, Uni..."


In [12]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (2772, 7)

Found 1 duplicates:
           city        country
2177  Palo Alto  United States

Resulting dataset: (2772, 7)
Merged artists with coordinates! Found 0 new locations
df_coordinates_concat exported to .csv


In [13]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
6700,Until Your Heart Stops,United States,Oakland,37.804456,-122.271356,"Oakland, Alameda County, California, United St..."
6701,Sleepercar,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
6702,Swamp Thing,New Zealand / Aotearoa,Rotorua,-38.138149,176.252922,"Rotorua, Rotorua Lakes District, Bay of Plenty..."
6703,Tim Kasher,United States,Omaha,41.258746,-95.938376,"Omaha, Douglas County, Nebraska, United States"
6704,Grateful Dead,United States,Palo Alto,37.444329,-122.159847,"Palo Alto, Santa Clara County, California, Uni..."


Later I found a null in Tableau, and after a lot of research I found an artist that was in ``df_artists_origins`` but for some reason didn't make it to ``df_artists_origins_coordinates``

In [ ]:
# import the last df that contains the coordinates of the unique origins
df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
df_artists_origins_coordinates = pd.merge(df_artists_origins, df_coordinates_concat, on=['origin'])

df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean'], inplace=True)
df_artists_origins_coordinates

,artist,country,city,latitude,longitude,address
0,Sparta,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
1,Muse,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
2,Finch,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
3,Transplants,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
4,Rooney,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
...,...,...,...,...,...,...
6701,Until Your Heart Stops,United States,Oakland,37.804456,-122.271356,"Oakland, Alameda County, California, United St..."
6702,Sleepercar,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
6703,Swamp Thing,New Zealand / Aotearoa,Rotorua,-38.138149,176.252922,"Rotorua, Rotorua Lakes District, Bay of Plenty..."
6704,Tim Kasher,United States,Omaha,41.258746,-95.938376,"Omaha, Douglas County, Nebraska, United States"


In [17]:
'Skywave' in df_artists_origins_coordinates['artist'].values

True

In [26]:
skywave = df_artists_origins_coordinates[df_artists_origins_coordinates['artist']=='Skywave']

In [ ]:
df = pd.concat([df_coordinates_final, skywave])


,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
1,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
2,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
3,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
4,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia
...,...,...,...,...,...,...,...
5629,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5630,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5631,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire
5632,No Devotion,United Kingdom,Pontypridd,51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ...",Rhondda Cynon Taf


In [32]:
df.reset_index(drop=True, inplace=True)
df

,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
1,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
2,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
3,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
4,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia
...,...,...,...,...,...,...,...
5629,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5630,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5631,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire
5632,No Devotion,United Kingdom,Pontypridd,51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ...",Rhondda Cynon Taf


In [33]:
df.to_csv('Datasets/df_coordinates_final.csv', index=False)

# **Statistics**

In [128]:
df_artists_origins_coordinates_concat = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')
df_artists_origins_coordinates_concat.shape

(6705, 6)

In [256]:
brittish_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United Kingdom']
american_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']

In [257]:
order = df_artists_origins_coordinates_concat['country'].value_counts().index
print(df_artists_origins_coordinates_concat['country'].value_counts().head())

# plt.figure(figsize=(8,6))
# sns.countplot(df_artists_origins_coordinates_concat['country'], order=order)

country
United States     4138
United Kingdom    1495
Canada             215
Sverige            156
Australia          133
Name: count, dtype: int64


In [258]:
print(f"{brittish_cities.shape[0]} Brittish artists")
order = brittish_cities['city'].value_counts().index
print(f"{brittish_cities['city'].nunique()} Brittish cities\n")
print(brittish_cities['city'].value_counts().head(15))

# plt.figure(figsize=(9,45))
# sns.countplot(brittish_cities['city'], order=order)

1495 Brittish artists
427 Brittish cities

city
London         404
Birmingham      40
Glasgow         40
Manchester      39
Brighton        39
Leeds           36
Liverpool       30
Bristol         26
Nottingham      22
Sheffield       19
Edinburgh       16
Cardiff         16
Cambridge       15
Reading         13
Southampton     13
Name: count, dtype: int64


In [259]:
print(f"{american_cities.shape[0]} American artists")
order = american_cities['city'].value_counts().index
print(f"{american_cities['city'].nunique()} American cities\n")
print(american_cities['city'].value_counts().head(15))

# plt.figure(figsize=(5,55))
# sns.countplot(df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']['city'], order=order)

4138 American artists
940 American cities

city
Los Angeles      367
New York City    189
Chicago          152
San Francisco    120
Seattle           99
Boston            97
Brooklyn          80
Philadelphia      71
Portland          70
San Diego         65
Atlanta           48
Austin            46
Washington        45
New York          41
Nashville         41
Name: count, dtype: int64


# **``state``**

I want to create a new column to analyze the distribution inside the country

In [260]:
df_artists_origins_coordinates_concat

,artist,country,city,latitude,longitude,address
0,Holy Moses,Deutschland,Aachen,51.132302,6.410734,"Wickrath Bahnhof, Ri. Aachen, Kohlenweg, Wickr..."
1,Mnemic,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
2,The Seven Mile Journey,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
3,Mercenary,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
4,Oh No Ono,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
...,...,...,...,...,...,...
6700,Wolf,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6701,Witchcraft,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6702,Truckfighters,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6703,No Devotion,United Kingdom,Pontypridd,51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ..."


In [261]:
df_origins = df_artists_origins_coordinates_concat

In [262]:
# separate the countries in 2 dataframes
df_artists_uk = df_origins[df_origins['country']=='United Kingdom']
df_artists_us = df_origins[df_origins['country']=='United States']

## **Testing**

### **United Kingdom**

In [51]:
# full address
df_artists_uk.iloc[0]['address']

'Teignmouth, Teignbridge, Devon, England, United Kingdom'

In [52]:
# I want to get the 3rd element starting from the tail, cause the 2nd one (England for instance) is too generic
df_artists_uk.iloc[0]['address'].split(', ')[-3]

'Devon'

In [42]:
df_artists_uk['address'].apply(lambda x: x.split(', ')[-2])

1                                     England
5                                     England
8                                     England
9                                     England
11                                    England
                        ...                  
6411                                  BN1 1HH
6421                                  England
6575                                  HP1 1EW
6588                                 RM17 6NP
6609    Northern Ireland / Tuaisceart Éireann
Name: address, Length: 1496, dtype: object

In [ ]:
# When I try to do the following it crashes
# df_artists_uk['address'].apply(lambda x: x.split(', ')[-3])

In [50]:
# create a new column, the length of the string 'address'
df_artists_uk['len_address'] = df_artists_uk['address'].str.len()

# look for short addresses
df_artists_uk.sort_values('len_address').head()

,artist,country,city,latitude,longitude,address,len_address
2974,Anderson Bruford Wakeman Howe,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
982,Pure Reason Revolution,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
2657,Blue Murder,United Kingdom,Inglaterra,52.531021,-1.264906,"England, United Kingdom",23
1162,Yndi Halda,United Kingdom,Kent,51.207075,0.721036,"Kent, England, United Kingdom",29
3654,Graphic Nature,United Kingdom,Kent,51.207075,0.721036,"Kent, England, United Kingdom",29


In [136]:
df_artists_uk_wrong = df_artists_uk.sort_values('len_address').head(3)
df_artists_uk_wrong

,artist,country,city,latitude,longitude,address,len_address
2974,Anderson Bruford Wakeman Howe,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
982,Pure Reason Revolution,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
2657,Blue Murder,United Kingdom,Inglaterra,52.531021,-1.264906,"England, United Kingdom",23


### **United States**

In [ ]:
# full address
df_artists_us.iloc[0]['address']

'El Paso, El Paso County, Texas, United States'

In [ ]:
# I want to get the 2nd element starting from the tail
df_artists_us.iloc[0]['address'].split(', ')[-2]

'Texas'

In [ ]:
# When I try to do the following it crashes
# df_artists_us['address'].apply(lambda x: x.split(', ')[-2])

In [ ]:
# create a new column, the length of the string 'address'
df_artists_us['len_address'] = df_artists_us['address'].str.len()

# look for short addresses
df_artists_us.sort_values('len_address').head()

,artist,country,city,latitude,longitude,address,len_address
2275,The Joe Perry Project,United States,United States,39.78373,-100.445882,United States,13
1075,Juliette and the Licks,United States,United States,39.78373,-100.445882,United States,13
934,Pride and Glory,United States,United States,39.78373,-100.445882,United States,13
2050,Blood Axis,United States,United States,39.78373,-100.445882,United States,13
4335,Nevermen,United States,United States,39.78373,-100.445882,United States,13


In [75]:
df_artists_us_wrong = df_artists_us[df_artists_us['city']=='United States']
df_artists_us_wrong.shape

(29, 7)

## **Change short ``address``**

In [137]:
df_artists_wrong = pd.concat([df_artists_us_wrong, df_artists_uk_wrong])
df_artists_wrong.shape

(32, 7)

In [130]:
# import dataframes
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_coordinates = pd.read_csv('Datasets/df_coordinates.csv')
df_artists_origins_coordinates = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# print information
print(f'{df_artists_origins.shape}: df_artists_origins')
print(f'{df_coordinates.shape}: df_coordinates')
print(f'{df_artists_origins_coordinates.shape}: df_artists_origins_coordinates')

(6738, 2): df_artists_origins
(2776, 7): df_coordinates
(6705, 6): df_artists_origins_coordinates


**Since I got this information from Wikipedia, I have to change these origins manually**

### **``np.where`` to replace the values for the real origins**

In [ ]:
for artist in df_artists_wrong['artist'].values:
    print(f'df_artists_wrong["origin"] = np.where(df_artists_wrong["artist"]=="{artist}", "States", df_artists_wrong["origin"])')

In [139]:
df_artists_wrong.iloc[20:]

,artist,country,city,latitude,longitude,address,len_address
4734,Unida,United States,United States,39.783730,-100.445882,United States,13
4826,Black Tape for a Blue Girl,United States,United States,39.783730,-100.445882,United States,13
5088,Dark New Day,United States,United States,39.783730,-100.445882,United States,13
5219,Heavens,United States,United States,39.783730,-100.445882,United States,13
5448,Charred Walls of the Damned,United States,United States,39.783730,-100.445882,United States,13
5558,I Can Make A Mess Like Nobody's Business,United States,United States,39.783730,-100.445882,United States,13
5680,Paths Of Possession,United States,United States,39.783730,-100.445882,United States,13
5939,Handsome,United States,United States,39.783730,-100.445882,United States,13
6002,Rock Star Supernova,United States,United States,39.783730,-100.445882,United States,13
2974,Anderson Bruford Wakeman Howe,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23


In [161]:
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Samhain", "Lodi, New Jersey", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Institute", "Brooklyn, NY, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Pride and Glory", "Los Angeles, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Juliette and the Licks", "Los Angeles, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Eyes Adrift", "Austin, TX, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Big Dumb Face", "Jacksonville, FL, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Goon Moon", "Los Angeles, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Superdrag", "Knoxville, Tennessee", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Blood Axis", "Denver, CO, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="OSI", "New York, NY, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Damn the Machine", "Los Angeles, California", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Nevermen", "Brooklyn, NY, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Planet X", "Los Angeles, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Chroma Key", "Kings Park, NY, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Unida", "Indio, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Black Tape for a Blue Girl", "Portland, OR, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Dark New Day", "Orlando, FL, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Heavens", "Los Angeles, CA, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Charred Walls of the Damned", "Long Island, New York", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="I Can Make A Mess Like Nobody's Business", "Hammonton, NJ, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Paths Of Possession", "Tampa, FL, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Handsome", "New York, NY, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Anderson Bruford Wakeman Howe", "London, England", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Pure Reason Revolution", "London, England", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Blue Murder", "Los Angeles, CA, United States", df_artists_origins["origin"])

In [156]:
# look for the albums of the artist in the original df_masters_blended to check it's the correct artist
df = df_masters_blended
df[df['artist']=="Blue Murder	".strip()].sort_values('year').head()

,master_id,artist,title,year,album_length,tracks,styles,artist_profile
2559,158074.0,Blue Murder,Blue Murder,1989,52.05,9,['Hard Rock'],"Blue Murder were an English hard rock band, fo..."


In [157]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[2559]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Blue Murder were an English hard rock band, founded by ex-Whitesnake, Tygers of Pan Tang and Thin Lizzy guitarist John
Sykes.


In [175]:
artists_to_remove = ['Monsters of Folk', 'Shrinebuilder', 'The Joe Perry Project', 'Boygenius',
                     'Scar the Martyr', 'Gordian Knot', 'Rock Star Supernova']

In [177]:
df_artists_corrected = df_artists_origins[(df_artists_origins['artist'].isin(df_artists_wrong['artist'].values)) & (~df_artists_origins['artist'].isin(artists_to_remove))]
df_artists_corrected.shape

(25, 2)

**Try all origins in Geopy**

In [ ]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 0
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

0 - Palo Alto, Santa Clara County, California, United States


### **Export to .csv**

In [240]:
df_artists_origins.to_csv('Datasets/df_artists_origins.csv', index=False)

In [ ]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(6738, 2)


In [242]:
df_artists_right = df_artists_origins[df_artists_origins['artist'].isin(['Demons and Wizards', 'No Devotion'])]
df_artists_right

,artist,origin
3823,No Devotion,"Pontypridd, Wales"
5342,Demons and Wizards,"Tampa, FL, United States"


### **GeoPy**

In [243]:
df_coordinates = get_coordinates_geopy(df_artists_right)
df_coordinates

1/2 - Pontypridd, Rhondda Cynon Taf, Cymru / Wales, United Kingdom
2/2 - Tampa, Hillsborough County, Florida, United States


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Pontypridd,"Pontypridd, Wales","Pontypridd, Wales",51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ..."
1,United States,Tampa,"Tampa, FL, United States","Tampa, FL, United States",27.947760,-82.458444,"Tampa, Hillsborough County, Florida, United St..."


In [244]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (2776, 7)

Found 2 duplicates:
            city         country
882   Pontypridd  United Kingdom
2553       Tampa   United States

Resulting dataset: (2776, 7)
Merged artists with coordinates! Found 0 new locations
df_coordinates_concat exported to .csv


In [245]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_artists_right)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
6702,Wolf,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6703,Witchcraft,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6704,Truckfighters,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6705,No Devotion,United Kingdom,Pontypridd,51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ..."
6706,Demons and Wizards,United States,Tampa,27.947760,-82.458444,"Tampa, Hillsborough County, Florida, United St..."


In [247]:
df_artists_origins_coordinates_concat.head()

,artist,country,city,latitude,longitude,address
0,Demons and Wizards,United States,\nGermany\nUnited States\n,40.420348,-79.116698,"Germany, West Wheatfield Township, Indiana Cou..."
1,No Devotion,United Kingdom,\nPontypridd,51.488309,-3.211589,"Lord Pontypridd House, Dulwich Gardens, Canton..."
2,Holy Moses,Deutschland,Aachen,51.132302,6.410734,"Wickrath Bahnhof, Ri. Aachen, Kohlenweg, Wickr..."
3,Mnemic,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
4,The Seven Mile Journey,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."


In [248]:
df_artists_origins_coordinates_concat.drop([0,1], axis=0, inplace=True)

In [249]:
df_artists_origins_coordinates_concat.shape

(6705, 6)

In [186]:
df_artists_origins_coordinates_concat['artist'].nunique()

6705

In [195]:
df_artists_origins_coordinates_concat.sort_values('city', inplace=True)
df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['artist'].duplicated()]

,artist,country,city,latitude,longitude,address
1128,Eyes Adrift,United States,United States,39.78373,-100.445882,United States
4398,Planet X,United States,United States,39.78373,-100.445882,United States
4568,Chroma Key,United States,United States,39.78373,-100.445882,United States
5680,Paths Of Possession,United States,United States,39.78373,-100.445882,United States
4826,Black Tape for a Blue Girl,United States,United States,39.78373,-100.445882,United States
1238,Big Dumb Face,United States,United States,39.78373,-100.445882,United States
4335,Nevermen,United States,United States,39.78373,-100.445882,United States
5558,I Can Make A Mess Like Nobody's Business,United States,United States,39.78373,-100.445882,United States
934,Pride and Glory,United States,United States,39.78373,-100.445882,United States
3550,Damn the Machine,United States,United States,39.78373,-100.445882,United States


In [193]:
df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['artist']=='Blue Murder']

,artist,country,city,latitude,longitude,address
2657,Blue Murder,United Kingdom,Inglaterra,52.531021,-1.264906,"England, United Kingdom"
6716,Blue Murder,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."


In [194]:
df_artists_origins_coordinates_concat.drop([2974,982,2657], axis=0, inplace=True)

In [250]:
check_duplicates_artists(df_artists_origins_coordinates_concat)

No duplicates


In [252]:
df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)
df_artists_origins_coordinates_concat

,artist,country,city,latitude,longitude,address
0,Holy Moses,Deutschland,Aachen,51.132302,6.410734,"Wickrath Bahnhof, Ri. Aachen, Kohlenweg, Wickr..."
1,Mnemic,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
2,The Seven Mile Journey,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
3,Mercenary,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
4,Oh No Ono,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
...,...,...,...,...,...,...
6700,Wolf,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6701,Witchcraft,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6702,Truckfighters,Sverige,Örebro,59.274729,15.215118,"Örebro, Örebro kommun, Örebro län, 703 61, Sve..."
6703,No Devotion,United Kingdom,Pontypridd,51.600105,-3.344936,"Pontypridd, Rhondda Cynon Taf, Cymru / Wales, ..."


In [253]:
df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)

In [234]:
df_artists_origins_coordinates_concat.head()

,artist,country,city,latitude,longitude,address
0,Demons and Wizards,United States,\nGermany\nUnited States\n,40.420348,-79.116698,"Germany, West Wheatfield Township, Indiana Cou..."
1,No Devotion,United Kingdom,\nPontypridd,51.488309,-3.211589,"Lord Pontypridd House, Dulwich Gardens, Canton..."
2,Holy Moses,Deutschland,Aachen,51.132302,6.410734,"Wickrath Bahnhof, Ri. Aachen, Kohlenweg, Wickr..."
3,Mnemic,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."
4,The Seven Mile Journey,Danmark,Aalborg,57.046263,9.921526,"Aalborg, Aalborg Kommune, Region Nordjylland, ..."


In [236]:
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="Demons and Wizards", "Tampa, FL, United States", df_artists_origins["origin"])
df_artists_origins["origin"] = np.where(df_artists_origins["artist"]=="No Devotion", "Pontypridd, Wales", df_artists_origins["origin"])

In [239]:
df_artists_origins

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."
...,...,...
6733,Until Your Heart Stops,"Oakland, CA, United States"
6734,Sleepercar,"El Paso, Texas, USA"
6735,Swamp Thing,"Rotorua, New Zealand"
6736,Tim Kasher,"Omaha, Nebraska, US"


## **Creating ``state``**

In [263]:
df_origins = df_artists_origins_coordinates_concat

In [264]:
# separate the countries in 2 dataframes
df_artists_uk = df_origins[df_origins['country']=='United Kingdom']
df_artists_us = df_origins[df_origins['country']=='United States']

### **United Kingdom**

In [265]:
# full address
df_artists_uk.iloc[0]['address']

'Aberdeen, Guild Street, City Centre, Aberdeen City, Alba / Scotland, AB11 6LX, United Kingdom'

In [266]:
# I want to get the 3rd element starting from the tail, cause the 2nd one (England for instance) is too generic
df_artists_uk.iloc[0]['address'].split(', ')[-3]

'Alba / Scotland'

In [279]:
df_artists_uk['state'] = df_artists_uk['address'].apply(lambda x: x.split(', ')[-3])
df_artists_uk

,artist,country,city,latitude,longitude,address,state
9,Korpse,United Kingdom,Aberdeen,57.142649,-2.097635,"Aberdeen, Guild Street, City Centre, Aberdeen ...",Alba / Scotland
11,Pallas,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
12,The Xcerts,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
13,Annie Lennox,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
14,Hellripper,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
...,...,...,...,...,...,...,...
6684,Asking Alexandria,United Kingdom,York,53.959055,-1.081536,"York, York and North Yorkshire, England, YO1 8...",England
6686,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
6687,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
6688,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire


### **United States**

In [269]:
# full address
df_artists_us.iloc[0]['address']

'Aberdeen, Grays Harbor County, Washington, 98520, United States'

In [270]:
# full address
df_artists_us.iloc[2]['address']

'Abilene, Taylor County, Texas, United States'

**the ``state`` is not always the same element, sometimes it's 2n from the tail, sometimes the 3rd**

In [271]:
# I want to get the 2nd element starting from the tail
df_artists_us.iloc[2]['address'].split(', ')[-3]

'Taylor County'

In [272]:
df_artists_us['address'].value_counts()

address
Los Angeles, Los Angeles County, California, United States    366
City of New York, New York, United States                     230
Chicago, Cook County, Illinois, United States                 152
San Francisco, California, United States                      120
Seattle, King County, Washington, United States                99
                                                             ... 
Kennesaw, Cobb County, Georgia, 30144, United States            1
Kennett, Dunklin County, Missouri, 63857, United States         1
Kent, King County, Washington, United States                    1
Kent, Portage County, Ohio, United States                       1
Zion, Lake County, Illinois, 60099, United States               1
Name: count, Length: 996, dtype: int64

In [273]:
import re

def extract_state(location):
    # Case 1: Full format with county and optional ZIP
    match = re.search(r'([\w\s]+),\s([\w\s]+),\s([\w\s]+)(?:,\s\d+)?,\sUnited States$', location)
    if match:
        return match.group(3)  # Extract state from 3rd capture group
    
    # Case 2: City and state only (no county, no ZIP)
    match = re.search(r'([\w\s]+),\s([\w\s]+),\sUnited States$', location)
    if match:
        return match.group(2)  # Extract state from 2nd capture group

    # Case 3: State only (e.g., "New Jersey, United States")
    match = re.search(r'([\w\s]+),\sUnited States$', location)
    if match:
        return match.group(1)  # Extract state from 1st capture group

    return None  # No match found

In [274]:
df_artists_us['state'] = df_artists_us['address'].apply(extract_state)

In [275]:
df_artists_us

,artist,country,city,latitude,longitude,address,state
10,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
18,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
19,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
21,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
23,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia
...,...,...,...,...,...,...,...
6685,Live,United States,York,39.962493,-76.727699,"York, York County, Pennsylvania, 17401, United...",Pennsylvania
6689,Bloodlined Calligraphy,United States,Ypsilanti,42.241051,-83.611765,"Ypsilanti, Washtenaw County, Michigan, United ...",Michigan
6692,Local H,United States,Zion,42.450117,-87.833775,"Zion, Lake County, Illinois, 60099, United States",Illinois
6697,Rifles at Recess,United States,chattanooga,35.045722,-85.309488,"Chattanooga, Hamilton County, East Tennessee, ...",Tennessee


In [298]:
df_artists_us['city'].value_counts().head(20)

city
Los Angeles      367
New York City    189
Chicago          152
San Francisco    120
Seattle           99
Boston            97
Brooklyn          80
Philadelphia      71
Portland          70
San Diego         65
Atlanta           48
Austin            46
Washington        45
New York          41
Nashville         41
Sacramento        38
Detroit           37
Oakland           37
Phoenix           35
Richmond          34
Name: count, dtype: int64

In [297]:
df_artists_us[df_artists_us['city'].str.contains('Franci')]

,artist,country,city,latitude,longitude,address,state
4541,Erase Errata,United States,Oakland/San Francisco,37.799388,-122.376576,"San Francisco–Oakland Bay Bridge West Span, Tr...",California
5431,Jefferson Starship,United States,San Francisco,37.779259,-122.419329,"San Francisco, California, United States",California
5432,4 Non Blondes,United States,San Francisco,37.779259,-122.419329,"San Francisco, California, United States",California
5433,Secret Chiefs 3,United States,San Francisco,37.779259,-122.419329,"San Francisco, California, United States",California
5434,The Organization,United States,San Francisco,37.779259,-122.419329,"San Francisco, California, United States",California
...,...,...,...,...,...,...,...
5553,Vicious Rumors,United States,San Francisco Bay Area,37.788497,-122.355847,"San Francisco Bay Area, San Francisco, Califor...",California
5554,Forbidden,United States,San Francisco Bay Area,37.788497,-122.355847,"San Francisco Bay Area, San Francisco, Califor...",California
5555,Vio-Lence,United States,San Francisco Bay Area,37.788497,-122.355847,"San Francisco Bay Area, San Francisco, Califor...",California
5556,Vain,United States,San Francisco Bay Area,37.788497,-122.355847,"San Francisco Bay Area, San Francisco, Califor...",California


## **Concat of both countries**

In [281]:
df_coordinates_final = pd.concat([df_artists_us, df_artists_uk])
df_coordinates_final

,artist,country,city,latitude,longitude,address,state
10,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
18,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
19,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
21,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
23,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia
...,...,...,...,...,...,...,...
6684,Asking Alexandria,United Kingdom,York,53.959055,-1.081536,"York, York and North Yorkshire, England, YO1 8...",England
6686,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
6687,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
6688,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire


In [282]:
df_coordinates_final.to_csv('Datasets/df_coordinates_final.csv', index=False)

# **``city_population``**

It's not really interesting if I analyze the cities with the most releases, because it's gonna be obviously the biggest cities.

In [20]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
df_ratings.head(1)

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.9,414,37.08,17,2.18,"['Ska', 'Punk']"


In [102]:
df_coordinates_final = pd.read_csv('Datasets/df_coordinates_final.csv')
df_coordinates_final.shape

(3994, 7)

In [3]:
df_coordinates_final.head()

,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
1,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
2,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
3,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
4,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia


## **United States**

In [23]:
pd.merge(df_coordinates_final, df_ratings, on='artist')[['country', 'city']].value_counts().head(10)

country         city         
United Kingdom  London           832
United States   Los Angeles      687
                New York City    450
                Chicago          281
                San Francisco    243
                Seattle          218
                Boston           216
United Kingdom  Birmingham       136
United States   San Diego        125
                Brooklyn         112
Name: count, dtype: int64

So I'm going to get the **population of the cities** so I can compare the amount of releases in relation to the size of the city, to discover musical hubs.

This time I'm not gonna scrape the data, I'm just going to get it from **Kaggle**.

In [ ]:
# keep only the columns I want
df_us_cities = df_coordinates_final.drop_duplicates(subset=['country', 'city'])
df_us_cities = df_us_cities[df_us_cities['country']=='United States']
df_us_cities.reset_index(drop=True, inplace=True)
df_us_cities = df_us_cities.loc[1:] # the first city was wrong, so I'm not gonna get it
df_us_cities = df_us_cities[['city', 'state']]
df_us_cities

,city,state
1,Abilene,Texas
2,Abington,Massachusetts
3,Adairsville,Georgia
4,Agoura Hills,California
5,Akron,Ohio
...,...,...
935,Yorba Linda,California
936,York,Pennsylvania
937,Ypsilanti,Michigan
938,Zion,Illinois


In [83]:
# dataset from Kaggle
uscities = pd.read_csv('Datasets/uscities.csv')
uscities.shape

(28338, 17)

In [85]:
uscities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [86]:
# keep only the columns I want
uscities = uscities[['city', 'state_name', 'population']].rename(columns={'state_name': 'state'})
uscities.head()

,city,state,population
0,New York,New York,18713220
1,Los Angeles,California,12750807
2,Chicago,Illinois,8604203
3,Miami,Florida,6445545
4,Dallas,Texas,5743938


In [86]:
df_us_population = pd.merge(df_us_cities, uscities, on=['city', 'state'])
df_us_population

,city,state,population
0,Abilene,Texas,123420
1,Adairsville,Georgia,4963
2,Agoura Hills,California,20222
3,Akron,Ohio,565208
4,Albany,New York,586383
...,...,...,...
740,Yonkers,New York,200370
741,Yorba Linda,California,67644
742,York,Pennsylvania,233184
743,Ypsilanti,Michigan,20171


In [ ]:
# sorting by the most populated cities
df_us_population = df_us_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_us_population

,city,state,population
0,New York,New York,18713220
1,Los Angeles,California,12750807
2,Chicago,Illinois,8604203
3,Miami,Florida,6445545
4,Dallas,Texas,5743938
...,...,...,...
740,Milford,Pennsylvania,982
741,Occidental,California,823
742,Upper Marlboro,Maryland,673
743,Gas,Kansas,501


In [ ]:
# export to csv
df_us_population.to_csv('Datasets/df_us_population.csv', index=False)

In [ ]:
df_us_population = pd.read_csv('Datasets/df_us_population.csv')
df_us_population.head(1)

,city,state,population
0,New York,New York,18713220


In [ ]:
# just in case there are hidden spaces
df_coordinates_final['city'] = df_coordinates_final['city'].str.strip()
df_us_population['city'] = df_us_population['city'].str.strip()

In [ ]:
# I do the merge with the lower versions, just in case some city doesn't match because of a capital letter
df_us_city_population = pd.merge(
    df_coordinates_final.assign(city_lower=df_coordinates_final['city'].str.lower().str.strip()),
    df_us_population.assign(city_lower=df_us_population['city'].str.lower().str.strip()), 
    on=['city_lower']).drop(columns=(['city_x', 'city_lower', 'latitude', 'longitude', 'address', 'state_x']))
df_us_city_population

,artist,country,city_y,state_y,population
0,Close Your Eyes,United States,Abilene,Texas,123420
1,Micah P. Hinson,United States,Abilene,Texas,123420
2,Pretty Boy Floyd,United States,Adairsville,Georgia,4963
3,Linkin Park,United States,Agoura Hills,California,20222
4,Foxygen,United States,Agoura Hills,California,20222
...,...,...,...,...,...
3668,Benjamin Francis Leftwich,United Kingdom,York,Pennsylvania,233184
3669,Glamour of the Kill,United Kingdom,York,Pennsylvania,233184
3670,Asking Alexandria,United Kingdom,York,Pennsylvania,233184
3671,The Seahorses,United Kingdom,York,Pennsylvania,233184


In [ ]:
# keep only artists from the US
df_us_city_population = df_us_city_population[df_us_city_population['country']=='United States'].drop(columns='country')
df_us_city_population.rename(columns=({'population_2020': 'population', 'state_y': 'state', 'city_y': 'city'}), inplace=True)
df_us_city_population = df_us_city_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_us_city_population

,artist,city,state,population
0,The Damned Things,New York,New York,18713220
1,Darwin Deez,New York,New York,18713220
2,Havohej,New York,New York,18713220
3,Zeal and Ardor,New York,New York,18713220
4,Alora Crucible,New York,New York,18713220
...,...,...,...,...
3541,Vanessa Carlton,Milford,Pennsylvania,982
3542,The Claypool Lennon Delirium,Occidental,California,823
3543,Dying Fetus,Upper Marlboro,Maryland,673
3544,009 Sound System,Gas,Kansas,501


In [ ]:
# export to csv
df_us_city_population.to_csv('Datasets/df_us_city_population.csv', index=False)

In [ ]:
# check there are no nulls
df_us_city_population.isna().sum()

artist        0
city          0
state         0
population    0
dtype: int64

In [ ]:
# smallest town I have
df_us_city_population['population'].min()

400

In [ ]:
df_us_city_population = pd.read_csv('Datasets/df_us_city_population.csv')
df_us_city_population.shape

(3546, 4)

For some reason I had problems in Tableau working with the 2 separate df merged, some nulls appeared out of nowhere when I had checked I had no nulls.

So I merged them here and tried with the merged df in Tableau, and I didn't have any problems, there were no null values anymore.

In [ ]:
df_final_us = pd.merge(df_final, df_us_city_population, on='artist')
df_final_us

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style,city,state,population
0,2007,!!!,Myth Takes,3.48,94,60.22,13,4.63,indie,Alternative Rock,Sacramento,California,1898019
1,2007,!!!,Myth Takes,3.48,94,60.22,13,4.63,indie,Indie Rock,Sacramento,California,1898019
2,2010,!!!,"Strange Weather, Isn't It?",3.06,41,40.80,9,4.53,indie,Indie Rock,Sacramento,California,1898019
3,1985,'Til Tuesday,Voices Carry,3.60,26,38.62,10,3.86,indie,New Wave,Boston,Massachusetts,4688346
4,1985,'Til Tuesday,Voices Carry,3.60,26,38.62,10,3.86,indie,Pop Rock,Boston,Massachusetts,4688346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10823,2001,pg.99,Document #7,3.80,226,25.42,5,5.08,punk,Punk,Sterling,Virginia,30271
10824,2001,pg.99,Document #8,4.22,916,27.12,9,3.01,indie,Emo,Sterling,Virginia,30271
10825,2001,pg.99,Document #8,4.22,916,27.12,9,3.01,punk,Hardcore,Sterling,Virginia,30271
10826,1997,that dog.,Retreat From the Sun,3.73,26,46.67,13,3.59,indie,Alternative Rock,Los Angeles,California,12750807


In [ ]:
df_final_us.to_csv('Datasets/df_final_us.csv', index=False)

## **United Kingdom**

Let's repeat the same steps with cities from the UK

In [36]:
df_uk_cities = df_coordinates_final.drop_duplicates(subset=['country', 'city'])
df_uk_cities = df_uk_cities[df_uk_cities['country']=='United Kingdom']
df_uk_cities.reset_index(drop=True, inplace=True)
df_uk_cities = df_uk_cities.loc[1:]
df_uk_cities = df_uk_cities[['city', 'state']]
df_uk_cities

,city,state
1,Abergele,Cymru / Wales
2,Abertawe,Cymru / Wales
3,Aberystwyth,Cymru / Wales
4,Abingdon,England
5,Accrington,England
...,...,...
422,Wrexham,Cymru / Wales
423,Wycombe,England
424,Yeovil,Somerset
425,York,York and North Yorkshire


In [32]:
kaggle_uk_cities = pd.read_excel('Datasets/kaggle_uk_cities.xlsx')
kaggle_uk_cities.shape

(2680, 9)

In [33]:
kaggle_uk_cities.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996


In [34]:
kaggle_uk_cities = kaggle_uk_cities[['country', 'city', 'population_proper']].rename(columns={'population_proper': 'population'})
kaggle_uk_cities.head()

,country,city,population
0,United Kingdom,London,8825001
1,United Kingdom,Birmingham,1137100
2,United Kingdom,Manchester,547627
3,United Kingdom,Leeds,789194
4,United Kingdom,Newcastle,879996


In [37]:
df_uk_population = pd.merge(df_uk_cities, kaggle_uk_cities, on=['city'])
df_uk_population

,city,state,country,population
0,Abergele,Cymru / Wales,United Kingdom,10577
1,Abertawe,Cymru / Wales,United Kingdom,245508
2,Aberystwyth,Cymru / Wales,United Kingdom,12250
3,Abingdon,England,United Kingdom,33130
4,Accrington,England,United Kingdom,35456
...,...,...,...,...
286,Worcester,Worcestershire,United Kingdom,101328
287,Worksop,England,United Kingdom,41820
288,Worthing,West Sussex,United Kingdom,109120
289,Yeovil,Somerset,United Kingdom,45000


In [38]:
df_uk_population = df_uk_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_uk_population

,city,state,country,population
0,London,Greater London,United Kingdom,8825001
1,Birmingham,West Midlands,United Kingdom,1137100
2,Newcastle,North East,United Kingdom,879996
3,Leeds,West Yorkshire,United Kingdom,789194
4,Glasgow,Alba / Scotland,United Kingdom,633120
...,...,...,...,...
286,Cuckfield,West Sussex,United Kingdom,3266
287,Preston,Lancashire,United Kingdom,3258
288,Icklesham,East Sussex,United Kingdom,2751
289,Meriden,West Midlands,United Kingdom,2719


In [39]:
df_uk_population.to_csv('Datasets/df_uk_population.csv', index=False)

In [ ]:
df_coordinates_final = pd.read_csv('Datasets/df_coordinates_final.csv')
df_coordinates_final.head(1)

,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington


In [121]:
df_coordinates_final['city'] = df_coordinates_final['city'].str.strip()
df_uk_population['city'] = df_uk_population['city'].str.strip()

In [122]:
df_uk_city_population = pd.merge(
    df_coordinates_final.assign(city_lower=df_coordinates_final['city'].str.lower().str.strip()),
    df_uk_population.assign(city_lower=df_uk_population['city'].str.lower().str.strip()), 
    on=['city_lower']).drop(columns=(['city_x', 'city_lower', 'latitude', 'longitude', 'address', 'state_x']))
df_uk_city_population

,artist,country,city_y,state_y,population
0,A Global Threat,United States,Bangor,Northern Ireland / Tuaisceart Éireann,61011
1,A Global Threat,United States,Bangor,Northern Ireland / Tuaisceart Éireann,16358
2,ERRA,United States,Birmingham,West Midlands,1137100
3,Pink Siifu,United States,Birmingham,West Midlands,1137100
4,Ghost Atlas,United States,Birmingham,West Midlands,1137100
...,...,...,...,...,...
1362,Glamour of the Kill,United Kingdom,York,York and North Yorkshire,153717
1363,Asking Alexandria,United Kingdom,York,York and North Yorkshire,153717
1364,The Seahorses,United Kingdom,York,York and North Yorkshire,153717
1365,Shed Seven,United Kingdom,York,York and North Yorkshire,153717


In [123]:
df_uk_city_population = df_uk_city_population[df_uk_city_population['country']=='United Kingdom'].drop(columns='country')
df_uk_city_population.rename(columns=({'state_y': 'state', 'city_y': 'city'}), inplace=True)
df_uk_city_population = df_uk_city_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_uk_city_population

,artist,city,state,population
0,Good Tiger,London,Greater London,8825001
1,The Psychedelic Furs,London,Greater London,8825001
2,Gryphon,London,Greater London,8825001
3,Thomas Dolby,London,Greater London,8825001
4,Big Deal,London,Greater London,8825001
...,...,...,...,...
1246,Xentrix,Preston,Lancashire,3258
1247,The Fireman,Icklesham,East Sussex,2751
1248,Blood From The Soul,Meriden,West Midlands,2719
1249,Napalm Death,Meriden,West Midlands,2719


In [124]:
df_uk_city_population.to_csv('Datasets/df_uk_city_population.csv', index=False)

In [ ]:
df_final_uk = pd.merge(df_final, df_uk_city_population, on='artist')
df_final_uk

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style,city,state,population
0,2007,*shels,Sea of the Dying Dhow,4.00,303,59.23,11,5.38,indie,Post Rock,London,Greater London,8825001
1,2007,*shels,Sea of the Dying Dhow,4.00,303,59.23,11,5.38,punk,Hardcore,London,Greater London,8825001
2,1973,10cc,10cc,3.75,30,35.23,10,3.52,classic,Art Rock,Stockport,Greater Manchester,105878
3,1974,10cc,Sheet Music,3.88,41,37.79,10,3.78,classic,Art Rock,Stockport,Greater Manchester,105878
4,1975,10cc,The Original Soundtrack,3.71,38,41.78,8,5.22,classic,Art Rock,Stockport,Greater Manchester,105878
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2007,iLiKETRAiNS,Elegies to Lessons Learnt,3.67,32,50.52,11,4.59,indie,Post Rock,Leeds,West Yorkshire,789194
3677,2004,yourcodenameis:milo,Ignoto,3.77,50,47.41,12,3.95,indie,Alternative Rock,Washington,England,67085
3678,2007,yourcodenameis:milo,They Came From the Sun,3.46,27,49.48,12,4.12,indie,Emo,Washington,England,67085
3679,2007,yourcodenameis:milo,They Came From the Sun,3.46,27,49.48,12,4.12,indie,Indie Rock,Washington,England,67085


In [ ]:
df_final_uk.to_csv('Datasets/df_final_uk.csv', index=False)

## **Both countries together**

In [41]:
df_uk_population = pd.read_csv('Datasets/df_uk_population.csv')
df_uk_population.head(1)

,city,state,country,population
0,London,Greater London,United Kingdom,8825001


In [ ]:
df_us_population = pd.read_csv('Datasets/df_us_population.csv')
df_us_population.head(1)

,city,state,population
0,New York,New York,18713220


In [43]:
df_us_population['country'] = 'United States'
df_us_population.head(1)

,city,state,population,country
0,New York,New York,18713220,United States


In [46]:
df_population = pd.concat([df_uk_population, df_us_population])
df_population.shape

(1036, 4)

In [49]:
df_population.to_csv('Datasets/df_population.csv', index=False)

In [ ]:
# I do the merge with the lower versions, just in case some city doesn't match because of a capital letter
df_all = pd.merge(
    df_coordinates_final.assign(city_lower=df_coordinates_final['city'].str.lower().str.strip()),
    df_population.assign(city_lower=df_population['city'].str.lower().str.strip()), 
    on=['city_lower']).drop(columns=(['city_x', 'city_lower', 'latitude', 'longitude', 'address', 'state_x']))
df_all

NameError: name 'df_us_population' is not defined

## Looking for cities in the UK that Tableau doesn't recognize

For some reason, Tableau doesn't recognize quite a lot of cities in the UK, maybe because I don't have more specific info like the state in the US.

Since correcting all of them (around 80) can take a long time, I create a subset to look for the cities with the most releases and correct only these ones in Tableau, in most of the cases introducing manually the coordinates.

In [4]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
df_ratings.head()

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,"['Ska', 'Punk']"
1,2002,Sparta,Wiretap Scars,3.79,431,47.62,12,3.97,"['Indie Rock', 'Alternative Rock']"
2,2003,Muse,Absolution,3.99,4411,52.18,14,3.73,['Alternative Rock']
3,1999,Muse,Showbiz,3.50,2181,49.65,12,4.14,"['Alternative Rock', 'Prog Rock']"
4,2002,Finch,What It Is to Burn,3.69,864,61.18,13,4.71,"['Emo', 'Post-Hardcore', 'Alternative Rock']"


In [5]:
df_artists_origins_coordinates[df_artists_origins_coordinates['city']=='Stanmore']

,artist,country,city,latitude,longitude,address
5930,Billy Idol,United Kingdom,Stanmore,51.618853,-0.302627,"Stanmore, London Road, Stanmore, London Boroug..."


In [102]:
df_country = pd.merge(df_ratings, df_artists_origins_coordinates, on='artist')
df_country_uk = df_country[df_country['country']=='United Kingdom']
df_country_uk.head(1)

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles,country,city,latitude,longitude,address
2,2003,Muse,Absolution,3.99,4411,52.18,14,3.73,['Alternative Rock'],United Kingdom,Teignmouth,50.546338,-3.49578,"Teignmouth, Teignbridge, Devon, England, Unite..."


In [103]:
df_country.shape

(11065, 14)

In [11]:
df_country_uk[df_country_uk['city']=='Wallsend']['artist'].value_counts()

artist
Sting    9
Name: count, dtype: int64

In [12]:
df_country_uk['city'].value_counts().head(60)

city
London                 832
Birmingham             136
Liverpool               87
Glasgow                 64
Manchester              60
Newcastle upon Tyne     41
Sheffield               40
Nottingham              34
Cardiff                 33
Leeds                   32
Belfast                 32
Bradford                31
Pinner                  27
Reading                 25
Blackpool               24
Hemel Hempstead         23
Canterbury              23
Oxford                  21
Brighton                21
Bristol                 21
Guildford               21
Coventry                20
Edinburgh               20
Barnsley                20
Prestwich               19
Dunfermline             19
Southampton             18
Ripley                  18
Ladbroke Grove          16
Aylesbury               16
Somerset                15
Salford                 15
Swindon                 15
Crawley                 15
Notting Hill            14
Wolverhampton           14
Southend-on-Sea        

In [13]:
df_country_uk.shape

(2833, 14)

In [14]:
df_country.shape

(11064, 14)

In [104]:
df_country.to_csv('Datasets/df_country.csv', index=False)

In [105]:
df_artist_countries = df_country[['artist', 'country']]
df_artist_countries.shape

(11065, 2)

In [106]:
df_artist_countries.to_csv('Datasets/df_artist_countries.csv', index=False)

In [61]:
df_final['artist'].nunique()

3995

In [63]:
df_artist_countries.head()

,artist,country
0,Less Than Jake,United States
1,Sparta,United States
2,Muse,United Kingdom
3,Muse,United Kingdom
4,Finch,United States


In [68]:
df_artist_countries.shape

(11064, 2)

In [65]:
df_artist_countries['artist'].nunique()

4135

In [70]:
df_artist_countries['artist'].value_counts().head()

artist
Elton John            27
Alice Cooper          25
The Rolling Stones    24
Bob Dylan             23
The Beach Boys        22
Name: count, dtype: int64

In [73]:
df_artist_countries[df_artist_countries['artist']=='Less Than Jake']

,artist,country
0,Less Than Jake,United States
72,Less Than Jake,United States
155,Less Than Jake,United States
431,Less Than Jake,United States
932,Less Than Jake,United States
1596,Less Than Jake,United States
2890,Less Than Jake,United States
6521,Less Than Jake,United States


In [74]:
df_final[df_final['artist']=='Less Than Jake']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style
7705,1995,Less Than Jake,Pezcore,3.73,191,50.07,21,2.38,punk,Punk
7706,1996,Less Than Jake,Losing Streak,3.90,414,37.08,17,2.18,punk,Punk
7707,1998,Less Than Jake,Hello Rockview,3.97,593,37.83,14,2.70,punk,Punk
7708,2003,Less Than Jake,Anthem,3.70,472,43.68,18,2.43,punk,Punk
7709,2004,Less Than Jake,B Is for B-sides,3.49,96,27.77,12,2.31,punk,Punk
7710,2006,Less Than Jake,In with the Out Crowd,2.67,279,39.02,12,3.25,indie,Pop Rock
7711,2006,Less Than Jake,In with the Out Crowd,2.67,279,39.02,12,3.25,punk,Punk
7712,2008,Less Than Jake,GNV FLA,3.62,238,37.85,15,2.52,punk,Punk


# **``state_population``**

It's not really interesting if I analyze the cities with the most releases, because it's gonna be obviously the biggest cities.

In [112]:
df_ratings = pd.read_csv('Datasets/df_ratings.csv')
df_ratings.head(1)

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
0,1996,Less Than Jake,Losing Streak,3.9,414,37.08,17,2.18,"['Ska', 'Punk']"


In [114]:
df_coordinates_final.head()

,artist,country,city,latitude,longitude,address,state
0,Annie Lennox,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
1,Pallas,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
2,The Xcerts,United Kingdom,Aberdeen,57.148243,-2.092809,"Aberdeen City, Alba / Scotland, United Kingdom",Aberdeen City
3,Jesu,United Kingdom,Abergele,53.284069,-3.581065,"Abergele, Conwy, Cymru / Wales, LL22 7BE, Unit...",Cymru / Wales
4,Warpath,United Kingdom,Abertawe,51.619596,-3.945925,"Swansea, Cymru / Wales, SA1 3BN, United Kingdom",Cymru / Wales


In [ ]:
pd.merge(df_coordinates_final, df_ratings, on='artist')[['country', 'city']].value_counts().head(10)

country         city         
United Kingdom  London           832
United States   Los Angeles      687
                New York City    450
                Chicago          281
                San Francisco    243
                Seattle          218
                Boston           216
United Kingdom  Birmingham       136
United States   San Diego        125
                Brooklyn         112
Name: count, dtype: int64

So I'm going to get the **population of the cities** so I can compare the amount of releases in relation to the size of the city, to discover musical hubs.

This time I'm not gonna scrape the data, I'm just going to get it from **Kaggle**.

## **Kaggle dataset with cities population**

In [80]:
# dataset from Kaggle
uscities_kaggle = pd.read_csv('Datasets/uscities.csv')
uscities_kaggle.shape

(28338, 17)

In [141]:
uscities_kaggle.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [142]:
kaggle_uk_cities = pd.read_excel('Datasets/kaggle_uk_cities.xlsx')
kaggle_uk_cities.shape

(2680, 9)

In [143]:
kaggle_uk_cities.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996


In [144]:
kaggle_uk_cities[kaggle_uk_cities['city'].isin(uscities['city'].values)]

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996
...,...,...,...,...,...,...,...,...,...
2654,Newington,51.3520,0.6679,United Kingdom,GB,Kent,NaN,2551,2551
2662,Broadway,52.0400,-1.8600,United Kingdom,GB,Worcestershire,NaN,2540,2540
2670,Moffat,55.3320,-3.4420,United Kingdom,GB,Dumfries and Galloway,NaN,2510,2510
2671,Stanton,52.3240,0.8850,United Kingdom,GB,Suffolk,NaN,2507,2507


There are cities in both countries with the same name, I have to be careful

In [145]:
uscities_kaggle[uscities_kaggle['city']=='London']

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
1299,London,London,KY,Kentucky,21125,Laurel,37.1175,-84.0767,37714,269,polygon,False,True,America/New_York,3,40744 40741,1840015258
3059,London,London,OH,Ohio,39097,Madison,39.8936,-83.4376,14870,475,polygon,False,True,America/New_York,3,43140,1840000674
11701,London,London,CA,California,6107,Tulare,36.4810,-119.4440,1690,1036,polygon,False,False,America/Los_Angeles,3,93618,1840017692
14273,London,London,AR,Arkansas,5115,Pope,35.3280,-93.2417,1067,129,polygon,False,True,America/Chicago,3,72847 72802,1840015402


### **Create a column of ``country``**

In [88]:
# keep only the columns I want
uscities = uscities_kaggle[['city', 'state_name', 'population']].rename(columns={'state_name': 'state'})
uscities.head()

,city,state,population
0,New York,New York,18713220
1,Los Angeles,California,12750807
2,Chicago,Illinois,8604203
3,Miami,Florida,6445545
4,Dallas,Texas,5743938


In [89]:
uscities['country'] = 'United States'
uscities.head(1)

,city,state,population,country
0,New York,New York,18713220,United States


### **Cities with the same name**

In [ ]:
# getting the cities in the US with at least one repetition
uscities['city'].value_counts()

city
Franklin         30
Fairview         23
Clinton          23
Marion           22
Madison          21
                 ..
Lebanon South     1
South Paris       1
Kiln              1
Pine Air          1
Monowi            1
Name: count, Length: 19090, dtype: int64

Luckily in the US I have info of the state, that way I can tell these cities with the same name apart

In [194]:
# getting the cities in the UK with at least one repetition
kaggle_uk_cities['city'].value_counts().head(50)

city
Wootton        4
Whittington    3
Bramley        3
Clifton        3
Broughton      3
Whitchurch     3
Newport        3
Preston        2
Blackburn      2
Hadleigh       2
Madeley        2
Bottesford     2
Haddenham      2
Gillingham     2
Southbourne    2
Huntington     2
Cheadle        2
Hatfield       2
Sudbury        2
Longton        2
Brampton       2
Kingswood      2
Wellington     2
Bangor         2
Southwick      2
Wollaston      2
Richmond       2
Swinton        2
Farnborough    2
Bilton         2
Hythe          2
Branston       2
Rothwell       2
Holywell       2
Ashford        2
Moulton        2
Hayes          2
Eccleston      2
Kenton         2
Willaston      2
Saint Ives     2
Nelson         2
Birstall       2
Cowley         2
Royston        2
Milton         2
Upton          2
Stonehouse     2
Stone          2
Treforest      1
Name: count, dtype: int64

Not in the UK, though...

In [ ]:
several_ukcities = kaggle_uk_cities['city'].value_counts().head(49).index
len(several_ukcities)

49

In [211]:
# search for cities in the UK with the same name that are the home of any of my artists
for city in several_ukcities:
    if city in df_coordinates_final[df_coordinates_final['country']=='United Kingdom']['city'].values:
        print(city)

Newport
Preston
Hatfield
Bangor
Rothwell


**Drop the cities I'm not interested in**

With the coordinates I can tell which city I want and which ones I can drop so I can properly merge the dataframes later

In [8]:
df_coordinates_final[df_coordinates_final['city']=='Rothwell'].query("country =='United Kingdom'")

,artist,country,city,latitude,longitude,address,state
694,The Pigeon Detectives,United Kingdom,Rothwell,53.749384,-1.478151,"Rothwell, Leeds, West Yorkshire, England, LS26...",England


In [210]:
kaggle_uk_cities[kaggle_uk_cities['city']=='Rothwell'].query("country =='United Kingdom'")

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
556,Rothwell,53.7485,-1.4780,United Kingdom,GB,Leeds,NaN,20354,20354
1363,Rothwell,52.4254,-0.8029,United Kingdom,GB,Northamptonshire,NaN,7694,7694


In [212]:
print(f"Rows before dropping: {kaggle_uk_cities.shape[0]}")
kaggle_uk_cities.drop([1363], axis=0, inplace=True)
print(f"Rows after dropping: {kaggle_uk_cities.shape[0]}")

Rows before dropping: 2675
Rows after dropping: 2674


In [215]:
kaggle_uk_cities.shape

(2674, 9)

In [216]:
# export to csv
kaggle_uk_cities.to_csv('Datasets/kaggle_uk_cities.csv', index=False)

In [213]:
df_coordinates_final.shape

(3995, 7)

In [217]:
# keep only the columns I want
ukcities = kaggle_uk_cities[['city', 'population', 'country']]
ukcities.head(1)

,city,population,country
0,London,10979000,United Kingdom


Since I don't have info of ``state`` in the UK cities, I have to do 2 merges separately and then concat them

In [56]:
df_uk = df_coordinates_final[df_coordinates_final['country']=='United Kingdom']
df_uk.shape

(828, 7)

In [251]:
df_uk_merged = pd.merge(df_coordinates_final, ukcities, on=['city', 'country'])
df_uk_merged.shape

(686, 8)

In [237]:
df_uk[~df_uk['artist'].isin(df_uk_merged['artist'].values)]

,artist,country,city,latitude,longitude,address,state
8,The Proclaimers,United Kingdom,Auchtermuchty,56.292085,-3.232828,"Auchtermuchty, Fife, Alba / Scotland, KY14 7AP...",Alba / Scotland
12,Marmaduke Duke,United Kingdom,Ayrshire,55.500000,-4.500000,"Ayrshire, Alba / Scotland, KA5 5NL, United Kin...",Alba / Scotland
22,Trigger the Bloodshed,United Kingdom,Bath/Bristol,51.386715,-2.402722,"Bristol & Bath Railway Path, Newbridge, Bath, ...",England
33,Druid,United Kingdom,Berkhamsted,51.760727,-0.565255,"Berkhamsted, Dacorum, Hertfordshire, England, ...",Hertfordshire
34,Peter Green,United Kingdom,Bethnal Green,51.530346,-0.056163,"Bethnal Green, London Borough of Tower Hamlets...",England
...,...,...,...,...,...,...,...
805,James,United Kingdom,Whalley Range,53.449363,-2.257469,"Whalley Range, Manchester, Greater Manchester,...",England
806,Brendan Perry,United Kingdom,Whitechapel,51.517486,-0.065968,"Whitechapel, London Borough of Tower Hamlets, ...",England
813,Al Stewart,United Kingdom,Wimborne,50.794537,-1.975959,"Wimborne, New Borough Road, St Johns, Wimborne...",England
816,Prefab Sprout,United Kingdom,Witton Gilbert,54.805942,-1.636224,"Witton Gilbert, County Durham, North East, Eng...",North East


**There are 142 brittish artists whose cities' population information I don’t have**

In [242]:
kaggle_uk_cities[kaggle_uk_cities['city']=='Witton Gilbert']

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper


In [77]:
df_us = df_coordinates_final[df_coordinates_final['country']=='United States']
df_us.shape

(3163, 7)

In [90]:
df_us_merged = pd.merge(df_coordinates_final, uscities, on=['city', 'country', 'state'])
df_us_merged.shape

(2714, 8)

In [91]:
missing_us_artists = df_us[~df_us['artist'].isin(df_us_merged['artist'].values)]
missing_us_artists.shape

(449, 7)

**There are 449 american artists whose cities' population information I don’t have**

In [265]:
missing_us_artists['city'].value_counts().head()

city
New York City    155
Long Island       29
Orange County     26
New Jersey        15
St Louis          12
Name: count, dtype: int64

In [270]:
uscities[uscities['city']=='Long Island']

,city,state,population,country
25650,Long Island,Kansas,121,United States


Not the Long Island in New York

In [286]:
df_coordinates_final[df_coordinates_final['city']=='New York'].shape

(26, 7)

In [287]:
df_coordinates_final[df_coordinates_final['city']=='New York City'].shape

(155, 7)

I have New York City under 2 different names

So there's no information from big cities like New Jersey, but there's information of boroughs like Bronx, Brooklyn, etc

In [284]:
df_coordinates_final[df_coordinates_final['city']=='New Jersey']

,artist,country,city,latitude,longitude,address,state
2604,Ashes Divide,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2605,Bedlight For Blue Eyes,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2606,Bigwig,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2607,Disciples of Mockery,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2608,Ducktails,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2609,Funebrarum,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2610,My Chemical Romance,United States,New Jersey,34.125423,-117.481312,"New Jersey, Morningside, Fontana, San Bernardi...",California
2611,Nokturnel,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2612,None More Black,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey
2613,Ours,United States,New Jersey,40.075738,-74.404162,"New Jersey, United States",New Jersey


So it turns out I can't do the merge on the city population because I have many missing values, I'll have to do it on the **state population**

In [300]:
uscities_kaggle[['city', 'state_name', 'population']].groupby('state_name').agg({'population': sum}).sort_values('population', ascending=False).head()

,population
state_name,
California,59652083
New York,36249123
Texas,33717239
Florida,31031088
Illinois,18104498


In [288]:
uscities_kaggle[uscities_kaggle['state_name']=='New York'].sort_values('population', ascending=False).head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
18,Brooklyn,Brooklyn,NY,New York,36047,Kings,40.6501,-73.9496,2559903,14221,polygon,False,True,America/New_York,1,11212 11213 11216 11233 11238 11209 11214 1122...,1840034030
19,Queens,Queens,NY,New York,36081,Queens,40.7498,-73.7976,2230722,7885,polygon,False,True,America/New_York,1,11361 11362 11363 11364 11354 11355 11356 1135...,1840034002
34,Manhattan,Manhattan,NY,New York,36061,New York,40.7834,-73.9662,1628706,27545,polygon,False,True,America/New_York,1,10026 10027 10030 10037 10039 10001 10011 1001...,1840034000
39,Bronx,Bronx,NY,New York,36005,Bronx,40.8501,-73.8662,1418207,12862,polygon,False,True,America/New_York,1,10453 10457 10460 10458 10467 10468 10451 1045...,1840033999


But I can't do a groupby, because there are boroughs inside cities, so I end up with larger populations than the real ones

## **Kaggle dataset with states population**

In [147]:
df_coordinates_final

,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington
1,Close Your Eyes,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
2,Micah P. Hinson,United States,Abilene,32.446450,-99.747591,"Abilene, Taylor County, Texas, United States",Texas
3,A Loss For Words,United States,Abington,42.104823,-70.945322,"Abington, Plymouth County, Massachusetts, 0235...",Massachusetts
4,Pretty Boy Floyd,United States,Adairsville,34.368327,-84.934906,"Adairsville, Bartow County, Georgia, 30103, Un...",Georgia
...,...,...,...,...,...,...,...
5628,Asking Alexandria,United Kingdom,York,53.959055,-1.081536,"York, York and North Yorkshire, England, YO1 8...",England
5629,The Seahorses,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5630,Shed Seven,United Kingdom,York,53.965658,-1.074305,"York, York and North Yorkshire, England, Unite...",York and North Yorkshire
5631,A Forest of Stars,United Kingdom,Yorkshire,53.982527,-1.385250,"Yorkshire, England, United Kingdom",Yorkshire


In [4]:
state_pops = pd.read_csv('Datasets/state_pops.csv')
state_pops.shape

(118, 52)

In [105]:
us_1990_pop = state_pops.iloc[90,1:].T.reset_index()
us_1990_pop.rename(columns=({'index': 'state', 90: 'state_pop'}), inplace=True)
us_1990_pop['state_pop'] = (us_1990_pop['state_pop']*1000000).astype(int)
us_1990_pop = us_1990_pop.sort_values('state').reset_index(drop=True)
us_1990_pop.head()

,state,state_pop
0,Alabama,4048508
1,Alaska,553120
2,Arizona,3679056
3,Arkansas,2354343
4,California,29950111


In [106]:
us_1990_pop

,state,state_pop
0,Alabama,4048508
1,Alaska,553120
2,Arizona,3679056
3,Arkansas,2354343
4,California,29950111
5,Colorado,3303862
6,Connecticut,3289055
7,D.C.,603814
8,Delaware,669063
9,Florida,13018365


In [117]:
pd.merge(df_coordinates_final, us_1990_pop, on='state').shape

(3160, 8)

In [108]:
df_us.shape

(3163, 7)

In [116]:
df_us[~df_us['artist'].isin(pd.merge(df_coordinates_final, us_1990_pop, on='state')['artist'].values)]

,artist,country,city,latitude,longitude,address,state
1041,El Grupo Nuevo de Omar Rodriguez Lopez,United States,Bayamón,18.344958,-66.168374,"Bayamón, Puerto Rico, United States",Puerto Rico
1788,Plan B,United States,Guayama,18.017959,-66.146502,"Guayama, Puerto Rico, United States",Puerto Rico
3184,Puya,United States,Puerto Rico,18.224771,-66.485829,"Puerto Rico, United States",Puerto Rico


So we basically left out Puerto Rico

In [ ]:
df_coordinates_us = pd.merge(df_coordinates_final, us_1990_pop, on='state')

In [142]:
df_coordinates_us[df_coordinates_us['city']=='Topeka']

,artist,country,city,latitude,longitude,address,state,state_pop
2986,Diskreet,United States,Topeka,39.049011,-95.677556,"Topeka, Shawnee County, Kansas, United States",Kansas,2480683
2987,Kansas,United States,Topeka,39.049011,-95.677556,"Topeka, Shawnee County, Kansas, United States",Kansas,2480683
2988,Origin,United States,Topeka,39.049011,-95.677556,"Topeka, Shawnee County, Kansas, United States",Kansas,2480683


In [136]:
df_coordinates_us.shape

(3160, 8)

In [137]:
df_exploded.shape

(16781, 10)

In [140]:
df_exploded.head(1)

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style
0,2007,!!!,Myth Takes,3.48,94,60.22,13,4.63,indie,Alternative Rock


In [50]:
merge = pd.merge(df_coordinates_final, df_exploded)
merge.shape

(16781, 16)

In [62]:
merge[merge['state']=='Washington']['style'].value_counts()

style
Alternative Rock     106
Indie Rock            88
Grunge                48
Punk                  45
Hardcore              34
Hard Rock             34
Heavy Metal           26
Doom Metal            23
Psychedelic Rock      22
Pop Rock              21
Emo                   16
Folk Rock             15
Stoner Rock           12
Post-Hardcore         11
Post Rock              9
Blues Rock             9
Pop Punk               7
Prog Rock              6
Metalcore              5
Country Rock           5
Thrash                 5
Death Metal            5
Black Metal            4
Soft Rock              3
Garage Rock            3
Progressive Metal      3
Classic Rock           3
Nu Metal               3
Art Rock               2
Rock & Roll            2
Post-Punk              1
Southern Rock          1
Name: count, dtype: int64

In [59]:
merge[merge['state']=='Washington'].query("year < 1991")['style'].count()

78

In [19]:
merge[merge['state']=='Idaho']['artist'].value_counts()

artist
Built to Spill    9
Josh Ritter       7
Treepeople        1
Name: count, dtype: int64

In [ ]:
merge2 = pd.merge(df_coordinates_us, df_exploded)
merge2.shape

(12500, 17)

In [122]:
df_ratings[df_ratings['artist']=='Pink Floyd'].sort_values('year')

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
75,1967,Pink Floyd,The Piper At The Gates Of Dawn,3.98,2858,41.92,11,3.81,"['Psychedelic Rock', 'Experimental']"
776,1968,Pink Floyd,A Saucerful Of Secrets,3.69,1835,39.40,7,5.63,['Psychedelic Rock']
1121,1969,Pink Floyd,Ummagumma,3.21,1569,86.58,11,7.87,"['Psychedelic Rock', 'Experimental', 'Prog Rock']"
598,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,"['Psychedelic Rock', 'Prog Rock', 'Symphonic R..."
499,1971,Pink Floyd,Meddle,4.17,3633,46.42,6,7.74,"['Psychedelic Rock', 'Prog Rock']"
991,1972,Pink Floyd,Obscured By Clouds,3.46,1456,40.48,10,4.05,"['Soundtrack', 'Psychedelic Rock', 'Prog Rock']"
54,1973,Pink Floyd,The Dark Side Of The Moon,4.55,8048,42.89,10,4.29,"['Prog Rock', 'Psychedelic Rock']"
85,1977,Pink Floyd,Animals,4.51,5420,41.52,5,8.30,['Prog Rock']
762,1979,Pink Floyd,The Wall,4.26,5917,80.99,26,3.12,"['Prog Rock', 'Art Rock']"
819,1983,Pink Floyd,The Final Cut,3.20,1764,43.23,12,3.60,"['Psychedelic Rock', 'Prog Rock']"


In [125]:
df_final_uk[df_final_uk['artist']=='Pink Floyd']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style,city,state,population
2039,1967,Pink Floyd,The Piper At The Gates Of Dawn,3.98,2858,41.92,11,3.81,classic,Psychedelic Rock,London,Greater London,8825001
2040,1968,Pink Floyd,A Saucerful Of Secrets,3.69,1835,39.40,7,5.63,classic,Psychedelic Rock,London,Greater London,8825001
2041,1969,Pink Floyd,Ummagumma,3.21,1569,86.58,11,7.87,classic,Prog Rock,London,Greater London,8825001
2042,1969,Pink Floyd,Ummagumma,3.21,1569,86.58,11,7.87,classic,Psychedelic Rock,London,Greater London,8825001
2043,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Prog Rock,London,Greater London,8825001
2044,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Psychedelic Rock,London,Greater London,8825001
2045,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Symphonic Rock,London,Greater London,8825001
2046,1971,Pink Floyd,Meddle,4.17,3633,46.42,6,7.74,classic,Prog Rock,London,Greater London,8825001
2047,1971,Pink Floyd,Meddle,4.17,3633,46.42,6,7.74,classic,Psychedelic Rock,London,Greater London,8825001
2048,1972,Pink Floyd,Obscured By Clouds,3.46,1456,40.48,10,4.05,classic,Prog Rock,London,Greater London,8825001


In [120]:
df_exploded[df_exploded['artist']=='Pink Floyd']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style
10206,1967,Pink Floyd,The Piper At The Gates Of Dawn,3.98,2858,41.92,11,3.81,classic,Psychedelic Rock
10207,1968,Pink Floyd,A Saucerful Of Secrets,3.69,1835,39.40,7,5.63,classic,Psychedelic Rock
10208,1969,Pink Floyd,Ummagumma,3.21,1569,86.58,11,7.87,classic,Prog Rock
10209,1969,Pink Floyd,Ummagumma,3.21,1569,86.58,11,7.87,classic,Psychedelic Rock
10210,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Prog Rock
10211,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Psychedelic Rock
10212,1970,Pink Floyd,Atom Heart Mother,3.59,2093,52.07,5,10.41,classic,Symphonic Rock
10213,1971,Pink Floyd,Meddle,4.17,3633,46.42,6,7.74,classic,Prog Rock
10214,1971,Pink Floyd,Meddle,4.17,3633,46.42,6,7.74,classic,Psychedelic Rock
10215,1972,Pink Floyd,Obscured By Clouds,3.46,1456,40.48,10,4.05,classic,Prog Rock


In [131]:
merge2[merge2['state']=='Florida']['style'].value_counts()

style
Death Metal          70
Alternative Rock     56
Punk                 50
Hardcore             42
Heavy Metal          40
Pop Rock             36
Southern Rock        32
Hard Rock            30
Emo                  28
Progressive Metal    23
Indie Rock           19
Nu Metal             17
Metalcore            17
Post-Hardcore        16
Blues Rock           16
Thrash               13
Pop Punk             12
Rock & Roll          10
Prog Rock             9
Classic Rock          7
Folk Rock             5
Soft Rock             5
Black Metal           4
Doom Metal            4
Post Rock             3
Grunge                3
Speed Metal           2
Psychedelic Rock      2
Stoner Rock           2
Symphonic Rock        2
Country Rock          2
Space Rock            1
Garage Rock           1
Shoegaze              1
Name: count, dtype: int64

In [130]:
merge2[merge2['state']=='Florida']['artist'].value_counts()

artist
The Allman Brothers Band           22
Tom Petty and the Heartbreakers    18
Lynyrd Skynyrd                     17
New Found Glory                    16
Savatage                           15
                                   ..
Automatic Loveletter                1
Dark New Day                        1
Black Tide                          1
Astronautalis                       1
Whole Wheat Bread                   1
Name: count, Length: 129, dtype: int64

In [103]:
df_coordinates_us[df_coordinates_us['state']=='Minnesota']['state_pop'].sum()

179878562

In [116]:
df_coordinates_us[df_coordinates_us['state']=='Washington']['state_pop'].sample()

2070    4900780
Name: state_pop, dtype: int64

In [113]:
4387282*2

8774564

In [117]:
(10/4900780)*1000000

2.040491513595795

In [21]:
df_ratings[df_ratings['artist']=='Bob Dylan']

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,styles
235,1965,Bob Dylan,Bringing It All Back Home,4.39,1149,46.53,11,4.23,"['Folk Rock', 'Folk']"
378,1966,Bob Dylan,Blonde on Blonde,4.41,1537,72.46,14,5.18,"['Folk Rock', 'Blues Rock']"
386,1965,Bob Dylan,Highway 61 Revisited,4.44,1890,48.82,9,5.42,"['Folk Rock', 'Blues Rock']"
404,1975,Bob Dylan,Blood on the Tracks,4.45,1586,51.67,10,5.17,"['Folk Rock', 'Acoustic', 'Ballad']"
445,1976,Bob Dylan,Desire,4.02,649,56.22,9,6.25,['Folk Rock']
688,1969,Bob Dylan,Nashville Skyline,3.79,539,26.63,10,2.66,"['Folk Rock', 'Country Rock']"
1680,1968,Bob Dylan,John Wesley Harding,3.89,537,38.52,12,3.21,"['Folk Rock', 'Country Rock']"
1681,1970,Bob Dylan,New Morning,3.44,295,35.45,12,2.95,"['Folk Rock', 'Country Rock']"
1682,1973,Bob Dylan,Dylan,1.74,177,33.15,9,3.68,['Folk Rock']
1683,1979,Bob Dylan,Slow Train Coming,3.05,228,46.65,9,5.18,"['Gospel', 'Rhythm & Blues', 'Vocal']"


For some reason I had problems in Tableau working with the 2 separate df merged, some nulls appeared out of nowhere when I had checked I had no nulls.

So I merged them here and tried with the merged df in Tableau, and I didn't have any problems, there were no null values anymore.

In [ ]:
df_final_us = pd.merge(df_final, df_us_city_population, on='artist')
df_final_us

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style,city,state,population
0,2007,!!!,Myth Takes,3.48,94,60.22,13,4.63,indie,Alternative Rock,Sacramento,California,1898019
1,2007,!!!,Myth Takes,3.48,94,60.22,13,4.63,indie,Indie Rock,Sacramento,California,1898019
2,2010,!!!,"Strange Weather, Isn't It?",3.06,41,40.80,9,4.53,indie,Indie Rock,Sacramento,California,1898019
3,1985,'Til Tuesday,Voices Carry,3.60,26,38.62,10,3.86,indie,New Wave,Boston,Massachusetts,4688346
4,1985,'Til Tuesday,Voices Carry,3.60,26,38.62,10,3.86,indie,Pop Rock,Boston,Massachusetts,4688346
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10823,2001,pg.99,Document #7,3.80,226,25.42,5,5.08,punk,Punk,Sterling,Virginia,30271
10824,2001,pg.99,Document #8,4.22,916,27.12,9,3.01,indie,Emo,Sterling,Virginia,30271
10825,2001,pg.99,Document #8,4.22,916,27.12,9,3.01,punk,Hardcore,Sterling,Virginia,30271
10826,1997,that dog.,Retreat From the Sun,3.73,26,46.67,13,3.59,indie,Alternative Rock,Los Angeles,California,12750807


In [ ]:
df_final_us.to_csv('Datasets/df_final_us.csv', index=False)

## **United Kingdom**

Let's repeat the same steps with cities from the UK

In [ ]:
kaggle_uk_cities = pd.read_excel('Datasets/kaggle_uk_cities.xlsx')
kaggle_uk_cities.shape

(2680, 9)

In [ ]:
kaggle_uk_cities.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996


In [ ]:
kaggle_uk_cities = kaggle_uk_cities[['country', 'city', 'population_proper']].rename(columns={'population_proper': 'population'})
kaggle_uk_cities.head()

,country,city,population
0,United Kingdom,London,8825001
1,United Kingdom,Birmingham,1137100
2,United Kingdom,Manchester,547627
3,United Kingdom,Leeds,789194
4,United Kingdom,Newcastle,879996


In [ ]:
df_uk_population = pd.merge(df_uk_cities, kaggle_uk_cities, on=['city'])
df_uk_population

,city,state,country,population
0,Abergele,Cymru / Wales,United Kingdom,10577
1,Abertawe,Cymru / Wales,United Kingdom,245508
2,Aberystwyth,Cymru / Wales,United Kingdom,12250
3,Abingdon,England,United Kingdom,33130
4,Accrington,England,United Kingdom,35456
...,...,...,...,...
286,Worcester,Worcestershire,United Kingdom,101328
287,Worksop,England,United Kingdom,41820
288,Worthing,West Sussex,United Kingdom,109120
289,Yeovil,Somerset,United Kingdom,45000


In [ ]:
df_uk_population = df_uk_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_uk_population

,city,state,country,population
0,London,Greater London,United Kingdom,8825001
1,Birmingham,West Midlands,United Kingdom,1137100
2,Newcastle,North East,United Kingdom,879996
3,Leeds,West Yorkshire,United Kingdom,789194
4,Glasgow,Alba / Scotland,United Kingdom,633120
...,...,...,...,...
286,Cuckfield,West Sussex,United Kingdom,3266
287,Preston,Lancashire,United Kingdom,3258
288,Icklesham,East Sussex,United Kingdom,2751
289,Meriden,West Midlands,United Kingdom,2719


In [ ]:
df_uk_population.to_csv('Datasets/df_uk_population.csv', index=False)

In [ ]:
df_coordinates_final = pd.read_csv('Datasets/df_coordinates_final.csv')
df_coordinates_final.head(1)

,artist,country,city,latitude,longitude,address,state
0,Nirvana,United States,Aberdeen,46.975371,-123.815722,"Aberdeen, Grays Harbor County, Washington, 985...",Washington


In [ ]:
df_coordinates_final['city'] = df_coordinates_final['city'].str.strip()
df_uk_population['city'] = df_uk_population['city'].str.strip()

In [ ]:
df_uk_city_population = pd.merge(
    df_coordinates_final.assign(city_lower=df_coordinates_final['city'].str.lower().str.strip()),
    df_uk_population.assign(city_lower=df_uk_population['city'].str.lower().str.strip()), 
    on=['city_lower']).drop(columns=(['city_x', 'city_lower', 'latitude', 'longitude', 'address', 'state_x']))
df_uk_city_population

,artist,country,city_y,state_y,population
0,A Global Threat,United States,Bangor,Northern Ireland / Tuaisceart Éireann,61011
1,A Global Threat,United States,Bangor,Northern Ireland / Tuaisceart Éireann,16358
2,ERRA,United States,Birmingham,West Midlands,1137100
3,Pink Siifu,United States,Birmingham,West Midlands,1137100
4,Ghost Atlas,United States,Birmingham,West Midlands,1137100
...,...,...,...,...,...
1362,Glamour of the Kill,United Kingdom,York,York and North Yorkshire,153717
1363,Asking Alexandria,United Kingdom,York,York and North Yorkshire,153717
1364,The Seahorses,United Kingdom,York,York and North Yorkshire,153717
1365,Shed Seven,United Kingdom,York,York and North Yorkshire,153717


In [ ]:
df_uk_city_population = df_uk_city_population[df_uk_city_population['country']=='United Kingdom'].drop(columns='country')
df_uk_city_population.rename(columns=({'state_y': 'state', 'city_y': 'city'}), inplace=True)
df_uk_city_population = df_uk_city_population.sort_values('population', ascending=False)\
                                    .reset_index(drop=True)
df_uk_city_population

,artist,city,state,population
0,Good Tiger,London,Greater London,8825001
1,The Psychedelic Furs,London,Greater London,8825001
2,Gryphon,London,Greater London,8825001
3,Thomas Dolby,London,Greater London,8825001
4,Big Deal,London,Greater London,8825001
...,...,...,...,...
1246,Xentrix,Preston,Lancashire,3258
1247,The Fireman,Icklesham,East Sussex,2751
1248,Blood From The Soul,Meriden,West Midlands,2719
1249,Napalm Death,Meriden,West Midlands,2719


In [ ]:
df_uk_city_population.to_csv('Datasets/df_uk_city_population.csv', index=False)

In [ ]:
df_final_uk = pd.merge(df_final, df_uk_city_population, on='artist')
df_final_uk

,year,artist,title,rating,votes,album_length,tracks,avg_song_length,subgenre,style,city,state,population
0,2007,*shels,Sea of the Dying Dhow,4.00,303,59.23,11,5.38,indie,Post Rock,London,Greater London,8825001
1,2007,*shels,Sea of the Dying Dhow,4.00,303,59.23,11,5.38,punk,Hardcore,London,Greater London,8825001
2,1973,10cc,10cc,3.75,30,35.23,10,3.52,classic,Art Rock,Stockport,Greater Manchester,105878
3,1974,10cc,Sheet Music,3.88,41,37.79,10,3.78,classic,Art Rock,Stockport,Greater Manchester,105878
4,1975,10cc,The Original Soundtrack,3.71,38,41.78,8,5.22,classic,Art Rock,Stockport,Greater Manchester,105878
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2007,iLiKETRAiNS,Elegies to Lessons Learnt,3.67,32,50.52,11,4.59,indie,Post Rock,Leeds,West Yorkshire,789194
3677,2004,yourcodenameis:milo,Ignoto,3.77,50,47.41,12,3.95,indie,Alternative Rock,Washington,England,67085
3678,2007,yourcodenameis:milo,They Came From the Sun,3.46,27,49.48,12,4.12,indie,Emo,Washington,England,67085
3679,2007,yourcodenameis:milo,They Came From the Sun,3.46,27,49.48,12,4.12,indie,Indie Rock,Washington,England,67085


In [ ]:
df_final_uk.to_csv('Datasets/df_final_uk.csv', index=False)

## **Both countries together**

In [ ]:
df_uk_population = pd.read_csv('Datasets/df_uk_population.csv')
df_uk_population.head(1)

,city,state,country,population
0,London,Greater London,United Kingdom,8825001


In [ ]:
df_us_population = pd.read_csv('Datasets/df_us_population.csv')
df_us_population.head(1)

,city,state,population
0,New York,New York,18713220


In [ ]:
df_us_population['country'] = 'United States'
df_us_population.head(1)

,city,state,population,country
0,New York,New York,18713220,United States


In [ ]:
df_population = pd.concat([df_uk_population, df_us_population])
df_population.shape

(1036, 4)

In [ ]:
df_population.to_csv('Datasets/df_population.csv', index=False)

In [ ]:
# I do the merge with the lower versions, just in case some city doesn't match because of a capital letter
df_all = pd.merge(
    df_coordinates_final.assign(city_lower=df_coordinates_final['city'].str.lower().str.strip()),
    df_population.assign(city_lower=df_population['city'].str.lower().str.strip()), 
    on=['city_lower']).drop(columns=(['city_x', 'city_lower', 'latitude', 'longitude', 'address', 'state_x']))
df_all

NameError: name 'df_us_population' is not defined